# Capstone Project

## Modeling Notebook - `PREPOST_dataset_COMBO` from 11/14/19

#### Importing Libraries

In [1]:
%matplotlib inline

# general libraries
import re
import string
import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing date libraries
import datetime as dt
import dateutil.parser as dparser

# scikit-learn libraries for preprocessing
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer

# scikit-learn libraries for constructing pipelines
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import clone

# scikit-learn libraries for clustering and dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.mixture import GaussianMixture

# scikit-learn libraries for evaluation
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# scikit-learn libraries for feature selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import RFECV

# scikit-learn libraries for learning
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_validate, cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# saving models
import pickle
from sklearn.externals import joblib

# setting pandas display options
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 10000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('precision', 5)
pd.options.mode.chained_assignment = None

/anaconda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


#### Directory/File Structure

In [2]:
sys.version

'3.6.9 |Anaconda custom (x86_64)| (default, Jul 30 2019, 13:42:17) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [3]:
print ('Running pandas version:', pd.__version__)
print ('Running numpy version:', np.__version__)
print ('Running sklearn version:', sklearn.__version__)

Running pandas version: 0.25.2
Running numpy version: 1.17.2
Running sklearn version: 0.21.3


In [4]:
os.getcwd()

'/Users/Shailesh/Desktop/MIDS/Capstone/w210CapstoneProject/notebooks'

In [5]:
os.chdir('../data')

In [6]:
sorted(os.listdir())

['.DS_Store',
 'Capstone_Fall_Shannon_Sept2019_request.csv',
 'POSTOP_ALL_col_names_11_9.pkl',
 'POSTOP_TREE_ALL_col_names_11_9.pkl',
 'POSTOP_categorical_TREE_col_names_11_14.pkl',
 'POSTOP_categorical_TREE_col_names_11_9.pkl',
 'POSTOP_categorical_TREE_col_names_12_1.pkl',
 'POSTOP_categorical_col_names_11_14.pkl',
 'POSTOP_categorical_col_names_11_9.pkl',
 'POSTOP_categorical_col_names_12_1.pkl',
 'POSTOP_numerical_col_names_11_14.pkl',
 'POSTOP_numerical_col_names_11_9.pkl',
 'POSTOP_numerical_col_names_12_1.pkl',
 'PREOP_ALL_col_names_11_9.pkl',
 'PREOP_TREE_ALL_col_names_11_9.pkl',
 'PREOP_categorical_TREE_col_names_11_14.pkl',
 'PREOP_categorical_TREE_col_names_11_9.pkl',
 'PREOP_categorical_TREE_col_names_12_1.pkl',
 'PREOP_categorical_col_names_11_14.pkl',
 'PREOP_categorical_col_names_11_9.pkl',
 'PREOP_categorical_col_names_12_1.pkl',
 'PREOP_dataset_10_24.pkl',
 'PREOP_dataset_TREE_10_24.pkl',
 'PREOP_dataset_TREE_cabg.pkl',
 'PREOP_dataset_TREE_cabvalve.pkl',
 'PREOP_datas

## Preprocessing Unprocessed Dataset

### Loading Dataset

#### `PREPOST_dataset_COMBO_11_14`

In [28]:
data = pd.read_pickle('../data/PREPOST_dataset_COMBO_12_1.pkl')

In [29]:
data.head(2)

age  heightcm  weightkg       bmi   hct  creatlst  totalbumin  a1clvl  \
0   54     180.0     117.0  36.11111  43.0       0.9         3.8     7.2   
1   65     175.3      79.4  25.83787  45.0       1.2         NaN     NaN   

   meldscr  hdef  pasys  surgdt_month_Jan  surgdt_month_Feb  surgdt_month_Mar  \
0      6.5  47.0   42.0                 0                 0                 0   
1      NaN  55.0   40.0                 0                 0                 0   

   surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  surgdt_month_Aug  \
0                 0                 0                 1                 0   
1                 0                 0                 1                 0   

   surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  surgdt_month_Dec  \
0                 0                 0                 0                 0   
1                 0                 0                 0                 0   

   surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  surgdt_DayOfWeek_Thurs  \
0                     0                      0                       0   
1                     0                      0                       0   

   surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  surgdt_DayOfWeek_Sun  \
0                     1                     0                     0   
1                     0                     1                     0   

   surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  surgdt_month  \
0                       1                       0             7   
1                       1                       0             7   

   surgdt_DayOfWeek surgdt_PartOfMonth  gender  racecaucasian  raceblack  \
0                 4                  1     1.0            1.0        0.0   
1                 5                  1     1.0            1.0        0.0   

   raceasian  racenativeam  racnativepacific  ethnicity  diabetes  dyslip  \
0        0.0           0.0               0.0        0.0       1.0     1.0   
1        0.0           0.0               0.0        0.0       0.0     1.0   

   dialysis  hypertn  infendo  slpapn  liverdis  immsupp  mediastrad  cancer  \
0       0.0        1      0.0     0.0       0.0        0         0.0     0.0   
1       0.0        1      0.0     0.0       0.0        0         0.0     0.0   

   pvd  syncope  unrespstat  cvd  cva  cvdtia  cvdpcarsurg  hitanti  prcvint  \
0  0.0      0.0         0.0  0.0  0.0     0.0          0.0      0.0      0.0   
1  0.0      0.0         0.0  0.0  0.0     0.0          0.0      0.0      1.0   

   prcab  prvalve  chf  priorhf  arrhyafib  medinotr  hdefd  vdaort  vdstena  \
0    0.0      0.0  0.0      0.0        0.0       0.0    1.0     1.0      0.0   
1    0.0      0.0  0.0      0.0        0.0       0.0    1.0     1.0      0.0   

   vdstenm  diabctrl  infendty  Tobacco_Combined  chrlungd  hmo2  ivdrugab  \
0      0.0       0.0       0.0               1.0       0.0   0.0       0.0   
1      0.0       0.0       0.0               0.0       0.0   0.0       0.0   

   alcohol  carshock24  resusc24  medasa  medaplt5days  medlipid  \
0      1.0         0.0       0.0     1.0             0       1.0   
1      0.0         0.0       0.0     1.0             0       0.0   

   numdisv_1_CORONARY  numdisv_2_CORONARIES  numdisv_3_CORONARIES  \
0                   0                     0                     1   
1                   0                     0                     1   

   anginalclass_STRENUOUS_ACTIVITY  anginalclass_SLIGHT_LIMITATION_ACTIVITY  \
0                                0                                        0   
1                                0                                        0   

   anginalclass_MARKED_LIMITATION_ACTIVITY  anginalclass_ANGINA_AT_REST  \
0                                        1                            0   
1                                        0                            1   

   classnyh_SLIGHT_LIMITATION  classnyh_MARKED_LIMITATION  \
0                           0                           0   
1        

In [30]:
data.shape

(42740, 254)

In [31]:
columns = data.columns

In [32]:
data= pd.read_csv('../dataWithProcedureType.csv')

In [33]:
data.shape

(42740, 263)

In [34]:
newColumns = data.columns
for col in newColumns:
    if col not in columns and col != 'procTyp':
        print(col)
        data.drop(col, axis=1, inplace=True)

mtopd
mtcause_Cardiac
mtcause_Infection
mtcause_Neuro
mtcause_Pulmonary
mtcause_Renal
mtcause_Vascular
mtcause


In [35]:
data.shape

(42740, 255)

In [36]:
data.head(2)

age  heightcm  weightkg       bmi   hct  creatlst  totalbumin  a1clvl  \
0   54     180.0     117.0  36.11111  43.0       0.9         3.8     7.2   
1   65     175.3      79.4  25.83787  45.0       1.2         NaN     NaN   

   meldscr  hdef  pasys  surgdt_month_Jan  surgdt_month_Feb  surgdt_month_Mar  \
0      6.5  47.0   42.0                 0                 0                 0   
1      NaN  55.0   40.0                 0                 0                 0   

   surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  surgdt_month_Aug  \
0                 0                 0                 1                 0   
1                 0                 0                 1                 0   

   surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  surgdt_month_Dec  \
0                 0                 0                 0                 0   
1                 0                 0                 0                 0   

   surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  surgdt_DayOfWeek_Thurs  \
0                     0                      0                       0   
1                     0                      0                       0   

   surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  surgdt_DayOfWeek_Sun  \
0                     1                     0                     0   
1                     0                     1                     0   

   surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  surgdt_month  \
0                       1                       0             7   
1                       1                       0             7   

   surgdt_DayOfWeek  surgdt_PartOfMonth  gender  racecaucasian  raceblack  \
0                 4                   1       1              1          0   
1                 5                   1       1              1          0   

   raceasian  racenativeam  racnativepacific  ethnicity  diabetes  dyslip  \
0          0             0                 0          0         1       1   
1          0             0                 0          0         0       1   

   dialysis  hypertn  infendo  slpapn  liverdis  immsupp  mediastrad  cancer  \
0         0        1        0       0         0        0           0       0   
1         0        1        0       0         0        0           0       0   

   pvd  syncope  unrespstat  cvd  cva  cvdtia  cvdpcarsurg  hitanti  prcvint  \
0    0        0           0    0    0       0            0        0        0   
1    0        0           0    0    0       0            0        0        1   

   prcab  prvalve  chf  priorhf  arrhyafib  medinotr  hdefd  vdaort  vdstena  \
0      0        0    0        0          0         0      1       1        0   
1      0        0    0        0          0         0      1       1        0   

   vdstenm  diabctrl  infendty  Tobacco_Combined  chrlungd  hmo2  ivdrugab  \
0        0         0         0                 1         0     0         0   
1        0         0         0                 0         0     0         0   

   alcohol  carshock24  resusc24  medasa  medaplt5days  medlipid  \
0        1           0         0       1             0         1   
1        0           0         0       1             0         0   

   numdisv_1_CORONARY  numdisv_2_CORONARIES  numdisv_3_CORONARIES  \
0                   0                     0                     1   
1                   0                     0                     1   

   anginalclass_STRENUOUS_ACTIVITY  anginalclass_SLIGHT_LIMITATION_ACTIVITY  \
0                                0                                        0   
1                                0                                        0   

   anginalclass_MARKED_LIMITATION_ACTIVITY  anginalclass_ANGINA_AT_REST  \
0                                        1                            0   
1                                        0                            1   

   classnyh_SLIGHT_LIMITATION  classnyh_MARKED_LIMITATION  \
0                           0                           0   
1     

In [37]:
cabg = data.loc[data['procTyp'] == 1]
valve = data.loc[data['procTyp'] == 2]
cabgValve = data.loc[data['procTyp'] == 3]

In [38]:
cabg.shape

(28945, 255)

In [39]:
valve.shape

(8500, 255)

In [40]:
cabgValve.shape

(5295, 255)

In [41]:
origData = data

In [42]:
data = cabgValve

### Dropping Irrelevant Columns

- `recordId`
- `cnstrokp`
- `cnstrokttia`
- `cncomaenceph`
- `strokeBin` as we are going to use the more inclusive `strokeBin2` as our outcome variable

#### Relative to prior versions, we are also going to retain `predstro` that will be used to compare our models versus the `STS` Model

In [43]:
cols_to_drop = ['recordId',
                'cnstrokp',
                'cnstrokttia',
                'cncomaenceph',
                'strokeBin',
               'procTyp']

In [44]:
len(cols_to_drop)

6

- dropping columns

In [45]:
data = data.drop(cols_to_drop, axis=1)

In [46]:
data.shape

(5295, 249)

- resetting `DataFrame` `index` 

In [47]:
data = data.reset_index(drop=True)

In [48]:
data.head()

age   heightcm  weightkg       bmi   hct  creatlst  totalbumin  a1clvl  \
0   54  180.00000     117.0  36.11111  43.0       0.9         3.8     7.2   
1   59  160.00000     127.5  49.80469  35.0       0.9         3.5     7.4   
2   56  165.10001      92.5  33.93498  37.0       1.1         4.3     6.3   
3   78  175.30000      74.8  24.34096  31.0       1.6         3.3     5.9   
4   80  157.50000      79.1  31.88712  37.0       1.7         NaN     5.7   

   meldscr  hdef  pasys  surgdt_month_Jan  surgdt_month_Feb  surgdt_month_Mar  \
0      6.5  47.0   42.0                 0                 0                 0   
1      6.4  60.0   35.0                 0                 0                 0   
2      7.3  65.0   33.0                 0                 0                 0   
3     11.0  65.0   34.0                 0                 0                 0   
4      NaN  53.0   57.0                 0                 0                 0   

   surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  surgdt_month_Aug  \
0                 0                 0                 1                 0   
1                 0                 0                 1                 0   
2                 0                 0                 1                 0   
3                 0                 0                 1                 0   
4                 0                 0                 1                 0   

   surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  surgdt_month_Dec  \
0                 0                 0                 0                 0   
1                 0                 0                 0                 0   
2                 0                 0                 0                 0   
3                 0                 0                 0                 0   
4                 0                 0                 0                 0   

   surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  surgdt_DayOfWeek_Thurs  \
0                     0                      0                       0   
1                     0                      1                       0   
2                     0                      1                       0   
3                     0                      0                       0   
4                     0                      1                       0   

   surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  surgdt_DayOfWeek_Sun  \
0                     1                     0                     0   
1                     0                     0                     0   
2                     0                     0                     0   
3                     1                     0                     0   
4                     0                     0                     0   

   surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  surgdt_month  \
0                       1                       0             7   
1                       1                       0             7   
2                       0                       0             7   
3                       0                       1             7   
4                       0                       1             7   

   surgdt_DayOfWeek  surgdt_PartOfMonth  gender  racecaucasian  raceblack  \
0                 4                   1       1              1          0   
1                 1                   1       0              1          0   
2                 1                   2       0              1          0   
3                 4                   3       1              1          0   
4                 1                   3       0              1          0   

   raceasian  racenativeam  racnativepacific  ethnicity  diabetes  dyslip  \
0          0             0                 0          0         1       1   
1          0             0                 0          0         1       0   
2          0             0                 0          0         1       0   
3          0             0                 0          0         0       1   
4   

In [49]:
data.tail()

age   heightcm  weightkg       bmi   hct  creatlst  totalbumin  a1clvl  \
5290   62  165.10001      77.6  28.46869  31.6      1.55         3.9     5.6   
5291   81  157.50000      62.5  25.19526  32.9      1.14         4.1     5.4   
5292   61  170.20000      80.3  27.72021  44.5      0.97         4.6     6.0   
5293   71  177.80000     126.0  39.85722  42.3      1.37         4.3     8.0   
5294   82  165.10001      74.9  27.47816  31.3      1.59         4.0     7.8   

      meldscr  hdef  pasys  surgdt_month_Jan  surgdt_month_Feb  \
5290    10.61  55.0    NaN                 0                 0   
5291     8.73  53.0    NaN                 0                 0   
5292     6.40  38.0   26.0                 0                 0   
5293    10.49  40.0   43.0                 0                 0   
5294    10.85  30.0    NaN                 0                 0   

      surgdt_month_Mar  surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  \
5290                 0                 0                 0                 0   
5291                 0                 0                 0                 0   
5292                 0                 0                 0                 0   
5293                 0                 0                 0                 0   
5294                 0                 0                 0                 0   

      surgdt_month_Aug  surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  \
5290                 0                 1                 0                 0   
5291                 0                 0                 1                 0   
5292                 0                 0                 0                 1   
5293                 0                 0                 0                 1   
5294                 0                 0                 0                 0   

      surgdt_month_Dec  surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  \
5290                 0                     0                      0   
5291                 0                     0                      0   
5292                 0                     1                      0   
5293                 0                     0                      0   
5294                 1                     0                      1   

      surgdt_DayOfWeek_Thurs  surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  \
5290                       0                     1                     0   
5291                       1                     0                     0   
5292                       0                     0                     0   
5293                       1                     0                     0   
5294                       0                     0                     0   

      surgdt_DayOfWeek_Sun  surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  \
5290                     0                       0                       1   
5291                     0                       0                       1   
5292                     0                       1                       0   
5293                     0                       1                       0   
5294                     0                       0                       0   

      surgdt_month  surgdt_DayOfWeek  surgdt_PartOfMonth  gender  \
5290             9                 4                   3       1   
5291            10                 3                   3       0   
5292            11                 0                   1       1   
5293            11                 3                   1       1   
5294            12                 1                   2       1   

      racecaucasian  raceblack  raceasian  racenativeam  racnativepacific  \
5290              1          0          0             0                 0   
5291              1          0          0             0                 0   
5292              0          0          0             0                 0   
5293              1          0          0             0                 0   
5294              1

In [50]:
data.shape

(5295, 249)

### Feature Matrix `X`
- in the next cell you can select which outcome vector you want to use: `strokeBin` or `strokeBin2`

In [51]:
X = data.copy().drop('strokeBin2', axis=1)

In [52]:
X = X.reset_index(drop=True)

In [53]:
X.head(2)

age  heightcm  weightkg       bmi   hct  creatlst  totalbumin  a1clvl  \
0   54     180.0     117.0  36.11111  43.0       0.9         3.8     7.2   
1   59     160.0     127.5  49.80469  35.0       0.9         3.5     7.4   

   meldscr  hdef  pasys  surgdt_month_Jan  surgdt_month_Feb  surgdt_month_Mar  \
0      6.5  47.0   42.0                 0                 0                 0   
1      6.4  60.0   35.0                 0                 0                 0   

   surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  surgdt_month_Aug  \
0                 0                 0                 1                 0   
1                 0                 0                 1                 0   

   surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  surgdt_month_Dec  \
0                 0                 0                 0                 0   
1                 0                 0                 0                 0   

   surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  surgdt_DayOfWeek_Thurs  \
0                     0                      0                       0   
1                     0                      1                       0   

   surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  surgdt_DayOfWeek_Sun  \
0                     1                     0                     0   
1                     0                     0                     0   

   surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  surgdt_month  \
0                       1                       0             7   
1                       1                       0             7   

   surgdt_DayOfWeek  surgdt_PartOfMonth  gender  racecaucasian  raceblack  \
0                 4                   1       1              1          0   
1                 1                   1       0              1          0   

   raceasian  racenativeam  racnativepacific  ethnicity  diabetes  dyslip  \
0          0             0                 0          0         1       1   
1          0             0                 0          0         1       0   

   dialysis  hypertn  infendo  slpapn  liverdis  immsupp  mediastrad  cancer  \
0         0        1        0       0         0        0           0       0   
1         0        1        0       0         0        0           0       0   

   pvd  syncope  unrespstat  cvd  cva  cvdtia  cvdpcarsurg  hitanti  prcvint  \
0    0        0           0    0    0       0            0        0        0   
1    1        0           0    0    0       0            0        0        0   

   prcab  prvalve  chf  priorhf  arrhyafib  medinotr  hdefd  vdaort  vdstena  \
0      0        0    0        0          0         0      1       1        0   
1      0        0    0        0          0         0      1       0        0   

   vdstenm  diabctrl  infendty  Tobacco_Combined  chrlungd  hmo2  ivdrugab  \
0        0         0         0                 1         0     0         0   
1        0         1         0                 0         0     0         0   

   alcohol  carshock24  resusc24  medasa  medaplt5days  medlipid  \
0        1           0         0       1             0         1   
1        0           0         0       0             0         1   

   numdisv_1_CORONARY  numdisv_2_CORONARIES  numdisv_3_CORONARIES  \
0                   0                     0                     1   
1                   1                     0                     0   

   anginalclass_STRENUOUS_ACTIVITY  anginalclass_SLIGHT_LIMITATION_ACTIVITY  \
0                                0                                        0   
1                                0                                        0   

   anginalclass_MARKED_LIMITATION_ACTIVITY  anginalclass_ANGINA_AT_REST  \
0                                        1                            0   
1                                        0                            0   

   classnyh_SLIGHT_LIMITATION  classnyh_MARKED_LIMITATION  \
0                           0                           0   
1     

In [54]:
X.shape

(5295, 248)

### Target Vector `y`

In [55]:
y = data.copy()['strokeBin2']

In [56]:
type(y)

pandas.core.series.Series

In [57]:
y.head(2)

0    0
1    0
Name: strokeBin2, dtype: int64

In [58]:
y.shape

(5295,)

In [59]:
y.unique()

array([0, 1])

In [60]:
y.value_counts()

0    5136
1     159
Name: strokeBin2, dtype: int64

### `X_train_all`, `X_devtest`, `y_train`, `y_devtest`
- using `train_test_split` with `stratify` parameter to ensure relative proportion of outcome classes are the same in `train`, `dev` and `test` sets
- observation split will be `80/10/10` between `train`, `dev`, `test`
- designation of `_all` denotes complete feature set (`PRE` + `POST`)

In [62]:
X_train_all, X_test_all, y_train, y_test = train_test_split(X,
                                                                  y,
                                                                  test_size=0.1,
                                                                  random_state=0,
                                                                  stratify=y)

#### validating `train_test_split`

In [63]:
X.shape

(5295, 248)

In [64]:
np.rint(X.shape[0] * 0.20)

1059.0

In [65]:
X_train_all.shape, X_test_all.shape, y_train.shape, y_test.shape

((4765, 248), (530, 248), (4765,), (530,))

In [67]:
X.shape[0] - X_train_all.shape[0] - X_test_all.shape[0]

0

In [68]:
y.shape[0] - y_train.shape[0] - y_test.shape[0]

0

#### validating `stratify` worked
- relative proportion of classes in `y`

In [69]:
print (np.round(y.value_counts()[0] / y.shape[0], 4))
print (np.round(y.value_counts()[1] / y.shape[0], 4))

0.97
0.03


- relative proportion of classes in `y_train`

In [70]:
print (np.round(y_train.value_counts()[0] / y_train.shape[0], 4))
print (np.round(y_train.value_counts()[1] / y_train.shape[0], 4))

0.97
0.03


- relative proportion of classes in `y_devtest`

In [72]:
print (np.round(y_test.value_counts()[0] / y_test.shape[0], 4))
print (np.round(y_test.value_counts()[1] / y_test.shape[0], 4))

0.9698
0.0302


### `X_dev_all`, `X_test_all`, `y_dev`, `y_test`

In [46]:
# X_dev_all, X_test_all, y_dev, y_test = train_test_split(X_devtest_all,
#                                                         y_devtest,
#                                                         test_size=0.5,
#                                                         random_state=0,
#                                                         stratify=y_devtest)

- validating `train_test_split`

In [47]:
# X_devtest_all.shape

(1059, 256)

In [48]:
# np.rint(X_devtest_all.shape[0] * 0.50)

530.0

In [49]:
# X_dev_all.shape, X_test_all.shape, y_dev.shape, y_dev.shape

((529, 256), (530, 256), (529,), (529,))

#### validating `stratify` worked
- relative proportion of classes in `y`

In [50]:
# print (np.round(y.value_counts()[0] / y.shape[0], 4))
# print (np.round(y.value_counts()[1] / y.shape[0], 4))

0.97
0.03


- relative proportion of classes in `y_devtest`

In [51]:
# print (np.round(y_devtest.value_counts()[0] / y_devtest.shape[0], 4))
# print (np.round(y_devtest.value_counts()[1] / y_devtest.shape[0], 4))

0.9698
0.0302


- relative proportion of classes in `y_dev`

In [52]:
# print (np.round(y_dev.value_counts()[0] / y_dev.shape[0], 4))
# print (np.round(y_dev.value_counts()[1] / y_dev.shape[0], 4))

0.9698
0.0302


- relative proportion of classes in `y_test`

In [53]:
# print (np.round(y_test.value_counts()[0] / y_test.shape[0], 4))
# print (np.round(y_test.value_counts()[1] / y_test.shape[0], 4))

0.9698
0.0302


### Resetting Indicies

In [73]:
X_train_all = X_train_all.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [55]:
# X_dev_all = X_dev_all.reset_index(drop=True)
# y_dev = y_dev.reset_index(drop=True)

In [74]:
X_test_all = X_test_all.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

### Extracting `predstro` from `X_train`, `X_dev` and `X_test`
- and subsequently creating `predstro_train`, `predstro_dev`, `predstro_test`

- `train`

In [75]:
X_train_all.shape

(4765, 248)

In [76]:
predstro_train = X_train_all['predstro']

In [77]:
predstro_train.shape

(4765,)

In [78]:
X_train_all = X_train_all.drop('predstro', axis=1)

In [79]:
X_train_all.head(2)

age   heightcm  weightkg       bmi   hct  creatlst  totalbumin  a1clvl  \
0   68  185.39999      84.6  24.61223  35.2       1.4         4.0     7.4   
1   61  180.30000      73.5  22.60976  47.0       0.8         4.7     5.7   

   meldscr  hdef  pasys  surgdt_month_Jan  surgdt_month_Feb  surgdt_month_Mar  \
0    12.57  48.0   81.0                 0                 0                 0   
1     6.40  55.0    NaN                 0                 0                 1   

   surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  surgdt_month_Aug  \
0                 0                 1                 0                 0   
1                 0                 0                 0                 0   

   surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  surgdt_month_Dec  \
0                 0                 0                 0                 0   
1                 0                 0                 0                 0   

   surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  surgdt_DayOfWeek_Thurs  \
0                     0                      0                       0   
1                     0                      0                       1   

   surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  surgdt_DayOfWeek_Sun  \
0                     0                     0                     0   
1                     0                     0                     0   

   surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  surgdt_month  \
0                       1                       0             5   
1                       0                       1             3   

   surgdt_DayOfWeek  surgdt_PartOfMonth  gender  racecaucasian  raceblack  \
0                 2                   1       1              1          0   
1                 3                   3       1              1          0   

   raceasian  racenativeam  racnativepacific  ethnicity  diabetes  dyslip  \
0          0             0                 0          0         1       1   
1          0             0                 0          0         0       0   

   dialysis  hypertn  infendo  slpapn  liverdis  immsupp  mediastrad  cancer  \
0         0        1        0       0         0        0           0       0   
1         0        0        0       0         0        0           0       0   

   pvd  syncope  unrespstat  cvd  cva  cvdtia  cvdpcarsurg  hitanti  prcvint  \
0    0        0           0    0    0       0            0        0        1   
1    0        0           0    0    0       0            0        0        1   

   prcab  prvalve  chf  priorhf  arrhyafib  medinotr  hdefd  vdaort  vdstena  \
0      0        0    1        1          0         1      1       0        0   
1      0        0    0        0          0         0      1       0        0   

   vdstenm  diabctrl  infendty  Tobacco_Combined  chrlungd  hmo2  ivdrugab  \
0        0         1         0                 1         0     0         0   
1        0         0         0                 0         0     0         0   

   alcohol  carshock24  resusc24  medasa  medaplt5days  medlipid  \
0        0           0         0       1             0         1   
1        0           0         0       1             0         1   

   numdisv_1_CORONARY  numdisv_2_CORONARIES  numdisv_3_CORONARIES  \
0                   0                     1                     0   
1                   0                     0                     1   

   anginalclass_STRENUOUS_ACTIVITY  anginalclass_SLIGHT_LIMITATION_ACTIVITY  \
0                                0                                        0   
1                                0                                        0   

   anginalclass_MARKED_LIMITATION_ACTIVITY  anginalclass_ANGINA_AT_REST  \
0                                        0                            0   
1                                        1                            0   

   classnyh_SLIGHT_LIMITATION  classnyh_MARKED_LIMITATION  \
0                           0                           0   
1  

In [80]:
X_train_all.shape

(4765, 247)

- `dev`

In [63]:
#X_dev_all.shape

(529, 256)

In [64]:
#predstro_dev = X_dev_all['predstro']

In [65]:
#predstro_dev.shape

(529,)

In [66]:
#X_dev_all = X_dev_all.drop('predstro', axis=1)

In [67]:
#X_dev_all.head(2)

age   heightcm  weightkg       bmi   hct  creatlst  totalbumin  a1clvl  \
0   85  178.00000      87.0  27.45865  40.8      0.67         3.5     5.7   
1   62  165.10001      75.4  27.66159  30.0      1.40         3.8     4.9   

   meldscr  hdef  pasys  surgdt_month_Jan  surgdt_month_Feb  surgdt_month_Mar  \
0     7.83  65.0   46.0                 0                 0                 0   
1    12.48  27.0   35.0                 0                 0                 0   

   surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  surgdt_month_Aug  \
0                 0                 0                 1                 0   
1                 1                 0                 0                 0   

   surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  surgdt_month_Dec  \
0                 0                 0                 0                 0   
1                 0                 0                 0                 0   

   surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  surgdt_DayOfWeek_Thurs  \
0                     0                      0                       0   
1                     1                      0                       0   

   surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  surgdt_DayOfWeek_Sun  \
0                     1                     0                     0   
1                     0                     0                     0   

   surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  surgdt_month  \
0                       0                       1             7   
1                       0                       0             4   

   surgdt_DayOfWeek  surgdt_PartOfMonth  gender  racecaucasian  raceblack  \
0                 4                   3       1              1          0   
1                 0                   2       1              1          0   

   raceasian  racenativeam  racnativepacific  ethnicity  diabetes  dyslip  \
0          0             0                 0          0         0       1   
1          0             0                 0          0         1       1   

   dialysis  hypertn  infendo  slpapn  liverdis  immsupp  mediastrad  cancer  \
0         0        1        0       0         0        0           0       0   
1         0        1        0       0         0        0           0       0   

   pvd  syncope  unrespstat  cvd  cva  cvdtia  cvdpcarsurg  hitanti  prcvint  \
0    0        0           0    1    1       0            0        0        0   
1    0        0           0    0    0       0            0        0        0   

   prcab  prvalve  chf  priorhf  arrhyafib  medinotr  hdefd  vdaort  vdstena  \
0      0        0    1        1          0         0      1       1        1   
1      0        0    1        1          0         0      1       1        1   

   vdstenm  diabctrl  infendty  Tobacco_Combined  chrlungd  hmo2  ivdrugab  \
0        0         0         0                 0         0     0         0   
1        0         0         0                 1         0     0         0   

   alcohol  carshock24  resusc24  medasa  medaplt5days  medlipid  \
0        0           0         0       1             0         1   
1        0           0         0       0             0         1   

   numdisv_1_CORONARY  numdisv_2_CORONARIES  numdisv_3_CORONARIES  \
0                   0                     0                     1   
1                   0                     0                     0   

   anginalclass_STRENUOUS_ACTIVITY  anginalclass_SLIGHT_LIMITATION_ACTIVITY  \
0                                0                                        0   
1                                0                                        0   

   anginalclass_MARKED_LIMITATION_ACTIVITY  anginalclass_ANGINA_AT_REST  \
0                                        1                            0   
1                                        0                            0   

   classnyh_SLIGHT_LIMITATION  classnyh_MARKED_LIMITATION  \
0                           1                           0   
1  

In [68]:
#X_dev_all.shape

(529, 255)

- `test`

In [81]:
X_test_all.shape

(530, 248)

In [82]:
predstro_test = X_test_all['predstro']

In [83]:
predstro_test.shape

(530,)

In [84]:
X_test_all = X_test_all.drop('predstro', axis=1)

In [85]:
X_test_all.head(2)

age  heightcm  weightkg       bmi   hct  creatlst  totalbumin  a1clvl  \
0   73     175.0      83.0  27.10204  38.0       1.8         3.7     5.7   
1   78     155.0      76.9  32.00833  38.0       1.1         4.1     7.3   

   meldscr  hdef  pasys  surgdt_month_Jan  surgdt_month_Feb  surgdt_month_Mar  \
0     16.6  60.0   55.0                 0                 0                 0   
1      7.3  68.0   34.0                 0                 0                 1   

   surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  surgdt_month_Aug  \
0                 0                 0                 0                 0   
1                 0                 0                 0                 0   

   surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  surgdt_month_Dec  \
0                 1                 0                 0                 0   
1                 0                 0                 0                 0   

   surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  surgdt_DayOfWeek_Thurs  \
0                     0                      1                       0   
1                     0                      1                       0   

   surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  surgdt_DayOfWeek_Sun  \
0                     0                     0                     0   
1                     0                     0                     0   

   surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  surgdt_month  \
0                       0                       0             9   
1                       0                       1             3   

   surgdt_DayOfWeek  surgdt_PartOfMonth  gender  racecaucasian  raceblack  \
0                 1                   2       1              1          0   
1                 1                   3       0              1          0   

   raceasian  racenativeam  racnativepacific  ethnicity  diabetes  dyslip  \
0          0             0                 0          0         0       1   
1          0             0                 0          0         1       1   

   dialysis  hypertn  infendo  slpapn  liverdis  immsupp  mediastrad  cancer  \
0         0        1        0       0         0        0           0       0   
1         0        1        0       0         0        0           0       0   

   pvd  syncope  unrespstat  cvd  cva  cvdtia  cvdpcarsurg  hitanti  prcvint  \
0    0        0           0    1    0       0            1        0        1   
1    0        0           0    1    1       0            0        0        0   

   prcab  prvalve  chf  priorhf  arrhyafib  medinotr  hdefd  vdaort  vdstena  \
0      0        0    0        1          1         0      1       0        0   
1      0        0    0        0          0         0      1       1        1   

   vdstenm  diabctrl  infendty  Tobacco_Combined  chrlungd  hmo2  ivdrugab  \
0        0         0         0                 1         0     0         0   
1        0         0         0                 0         0     0         0   

   alcohol  carshock24  resusc24  medasa  medaplt5days  medlipid  \
0        0           0         0       1             0         1   
1        0           0         0       1             0         1   

   numdisv_1_CORONARY  numdisv_2_CORONARIES  numdisv_3_CORONARIES  \
0                   0                     1                     0   
1                   0                     1                     0   

   anginalclass_STRENUOUS_ACTIVITY  anginalclass_SLIGHT_LIMITATION_ACTIVITY  \
0                                0                                        1   
1                                0                                        0   

   anginalclass_MARKED_LIMITATION_ACTIVITY  anginalclass_ANGINA_AT_REST  \
0                                        0                            0   
1                                        0                            0   

   classnyh_SLIGHT_LIMITATION  classnyh_MARKED_LIMITATION  \
0                           0                           0   
1     

In [86]:
X_test_all.shape

(530, 247)

- validating row count - `42,740` total observations

In [87]:
42740 - X_train_all.shape[0]  - X_test_all.shape[0]#- X_dev_all.shape[0]

37445

#### Last look at the data (`X_train_all`) before modeling

In [88]:
X_train_all.head()

age   heightcm  weightkg       bmi   hct  creatlst  totalbumin  a1clvl  \
0   68  185.39999      84.6  24.61223  35.2       1.4         4.0     7.4   
1   61  180.30000      73.5  22.60976  47.0       0.8         4.7     5.7   
2   62  147.00000      51.0  23.60128  33.0       1.1         4.2     6.1   
3   85  193.00000     102.0  27.38329  51.0       1.4         4.3     5.3   
4   68  160.00000      62.8  24.53125  38.0       0.9         4.0     5.0   

   meldscr  hdef  pasys  surgdt_month_Jan  surgdt_month_Feb  surgdt_month_Mar  \
0    12.57  48.0   81.0                 0                 0                 0   
1     6.40  55.0    NaN                 0                 0                 1   
2     7.30  30.0   37.0                 0                 0                 1   
3     9.63  60.0   35.0                 0                 0                 1   
4     7.50  55.0   37.0                 0                 0                 0   

   surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  surgdt_month_Aug  \
0                 0                 1                 0                 0   
1                 0                 0                 0                 0   
2                 0                 0                 0                 0   
3                 0                 0                 0                 0   
4                 0                 0                 0                 0   

   surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  surgdt_month_Dec  \
0                 0                 0                 0                 0   
1                 0                 0                 0                 0   
2                 0                 0                 0                 0   
3                 0                 0                 0                 0   
4                 0                 0                 0                 1   

   surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  surgdt_DayOfWeek_Thurs  \
0                     0                      0                       0   
1                     0                      0                       1   
2                     0                      0                       0   
3                     1                      0                       0   
4                     0                      1                       0   

   surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  surgdt_DayOfWeek_Sun  \
0                     0                     0                     0   
1                     0                     0                     0   
2                     0                     0                     0   
3                     0                     0                     0   
4                     0                     0                     0   

   surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  surgdt_month  \
0                       1                       0             5   
1                       0                       1             3   
2                       0                       0             3   
3                       0                       0             3   
4                       0                       0            12   

   surgdt_DayOfWeek  surgdt_PartOfMonth  gender  racecaucasian  raceblack  \
0                 2                   1       1              1          0   
1                 3                   3       1              1          0   
2                 2                   2       0              1          0   
3                 0                   2       1              1          0   
4                 1                   2       1              1          0   

   raceasian  racenativeam  racnativepacific  ethnicity  diabetes  dyslip  \
0          0             0                 0          0         1       1   
1          0             0                 0          0         0       0   
2          0             0                 0          0         0       1   
3          0             0                 0          0         0       1   
4   

In [89]:
X_train_all.tail()

age   heightcm  weightkg       bmi   hct  creatlst  totalbumin  a1clvl  \
4760   74  165.10001      78.8  28.90893  31.0       1.1         4.0     6.7   
4761   70  173.00000     104.0  34.74891  41.0       1.2         NaN     6.1   
4762   54  178.00000      80.7  25.47027  44.0       1.0         NaN     5.5   
4763   84  165.10001      81.1  29.75272  33.4       1.0         4.0     NaN   
4764   80  177.80000      84.0  26.57148  26.0       1.1         4.0     6.2   

      meldscr  hdef  pasys  surgdt_month_Jan  surgdt_month_Feb  \
4760      8.2   NaN    NaN                 0                 0   
4761      NaN  60.0   34.0                 0                 1   
4762      NaN  43.0    NaN                 0                 0   
4763      NaN  53.0   44.0                 0                 0   
4764     14.5  75.0   44.0                 0                 0   

      surgdt_month_Mar  surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  \
4760                 0                 0                 0                 0   
4761                 0                 0                 0                 0   
4762                 0                 0                 0                 0   
4763                 0                 1                 0                 0   
4764                 0                 0                 0                 1   

      surgdt_month_Aug  surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  \
4760                 0                 0                 0                 0   
4761                 0                 0                 0                 0   
4762                 0                 0                 0                 0   
4763                 0                 0                 0                 0   
4764                 0                 0                 0                 0   

      surgdt_month_Dec  surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  \
4760                 1                     0                      0   
4761                 0                     0                      0   
4762                 1                     0                      1   
4763                 0                     0                      0   
4764                 0                     0                      0   

      surgdt_DayOfWeek_Thurs  surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  \
4760                       1                     0                     0   
4761                       0                     0                     0   
4762                       0                     0                     0   
4763                       0                     0                     1   
4764                       0                     0                     0   

      surgdt_DayOfWeek_Sun  surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  \
4760                     0                       0                       1   
4761                     0                       1                       0   
4762                     0                       1                       0   
4763                     0                       1                       0   
4764                     0                       0                       1   

      surgdt_month  surgdt_DayOfWeek  surgdt_PartOfMonth  gender  \
4760            12                 3                   3       0   
4761             2                 2                   1       1   
4762            12                 1                   1       1   
4763             4                 5                   1       0   
4764             7                 2                   3       1   

      racecaucasian  raceblack  raceasian  racenativeam  racnativepacific  \
4760              1          0          0             0                 0   
4761              1          0          0             0                 0   
4762              1          0          0             0                 0   
4763              1          0          0             0                 0   
4764              1

## Pre-processing Data using `Scikit-Learn` `Pipelines`

### Creating a `FeatureSelector` transformer
- necessary because we are working with heterogeneous data (numerical and categorical features) -- want to be able to pick and choose which features (columns) to pass through our pipelines (and transform them) instead of having to pass through the whole dataframe

In [90]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    '''
    Transformer to select column from a data frame to perform additional transformations on
    Use this for selecting column(s) that require fit transform
    '''
    
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[data_dict.columns.intersection(self.key)]

## Loading Column Names for `PREOP` and `POSTOP` Features
- these `lists` of features will be used to mask/select specified features from the `COMBO` dataset to assemble `PREOP`, `PREOP_TREE`, `POSTOP` and `POSTOP_TREE` feature sets as appropriate
- these are essentially helper lists as `DataFrames` put through `Pipelines` emerge as `numpy` arrays which do not have column names
- can convert resulting `numpy` arrays back into `DataFrames` for further analysis

#### `PREOP_dataset` Column Names

- `DUMMIES` Version = `PREOP_numerical_col_names` + `PREOP_categorical_col_names`
- `TREE` Version    = `PREOP_numerical_col_names` + `PREOP_categorical_TREE_col_names`

- `PREOP_numerical_col_names`

In [91]:
with open('PREOP_numerical_col_names_12_1.pkl', 'rb') as filehandle:
    PREOP_numerical_col_names = pickle.load(filehandle)

In [92]:
PREOP_numerical_col_names[0:5]

['age', 'heightcm', 'weightkg', 'bmi', 'hct']

In [93]:
len(PREOP_numerical_col_names)

11

- `PREOP_categorical_col_names`

In [94]:
with open('PREOP_categorical_col_names_12_1.pkl', 'rb') as filehandle:
    PREOP_categorical_col_names = pickle.load(filehandle)

In [95]:
PREOP_categorical_col_names[0:5]

['surgdt_month_Jan',
 'surgdt_month_Feb',
 'surgdt_month_Mar',
 'surgdt_month_Apr',
 'surgdt_month_May']

In [96]:
len(PREOP_categorical_col_names)

99

- `PREOP_categorical_TREE_col_names`

In [97]:
with open('PREOP_categorical_TREE_col_names_12_1.pkl', 'rb') as filehandle:
    PREOP_categorical_TREE_col_names = pickle.load(filehandle)

In [98]:
PREOP_categorical_TREE_col_names[0:5]

['surgdt_month',
 'surgdt_DayOfWeek',
 'surgdt_PartOfMonth',
 'gender',
 'racecaucasian']

In [99]:
len(PREOP_categorical_TREE_col_names)

61

#### `POSTOP_dataset` Column Names

- `DUMMIES` Version = `POSTOP_numerical_col_names` + `POSTOP_categorical_col_names`
- `TREE` Version = `POSTOP_numerical_col_names` + `POSTOP_categorical_TREE_col_names`

In [100]:
with open('POSTOP_numerical_col_names_12_1.pkl', 'rb') as filehandle:
    POSTOP_numerical_col_names = pickle.load(filehandle)

In [101]:
with open('POSTOP_categorical_col_names_12_1.pkl', 'rb') as filehandle:
    POSTOP_categorical_col_names = pickle.load(filehandle)

In [102]:
with open('POSTOP_categorical_TREE_col_names_12_1.pkl', 'rb') as filehandle:
    POSTOP_categorical_TREE_col_names = pickle.load(filehandle)

## Functions Used in Pipelines

#### `SimpleImputer(missing_values=np.nan, strategy='median'` 
- to replace numerical feature `NaN`s with `X_train` numerical feature `NaN`s

#### `PolynomialFeatures()`
- per Albon 13.2 pages 225-227, can use `PolynomialFeatures()` to create interaction terms
- Parameter `interaction_only=True` tells `PolynomialFeatures()` to ONLY return interaction terms (and not polynomial features
- By default, `PolynomialFeatures()` will add a feature containing ones (a vector of `1`s) called a `bias` -- we can prevent that through the parameter `include_bias=False`
- The `degree` parameter determines the maximum number of features to create interaction terms from (in case we wanted to create an interaction term of `n_features`)
- open question whether we should apply `PolynomialFeatures` to the entire feature matrix or only to the numerical and/or categorical features and how we should do it (with everything higher degree + interaction terms or just interaction terms).
- Given the number of featues we have, we will have many more features after transforming with `PolynomialFeatures`
- The formula for calculating the number of the polynomial features is `N(n,d)=C(n+d,d)` where `n` is the number of the features, `d` is the degree of the polynomial, `C` is binomial coefficient(combination). In our case the number is `C(3+2,2)=5!/(5-2)!2!=10` but when the number of features or the degree is high the polynomial features becomes too many.

#### `sklearn.preprocessing` scalers such as `StandardScaler()`

#### Custom `helper` functions, such as `convert_df_to_numpy`
- need to create a function to convert `DataFrame` to `numpy.ndarray` then can use `FeatureUnion` to combine with `numpy.ndarray` that results from `numerical_pipeline` to form feature matrix

In [103]:
def convert_df_to_numpy(df):
    
    df = df.values
    
    return df

#### `FeatureUnion`
- to reassemble feature matrix from the product of multiple `pipelines`

# `PREOP` Feature Matrix Pipelines

### `PREOP_numerical_features` Pipeline

In [104]:
PREOP_numerical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(PREOP_numerical_col_names)), 
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('scaler', StandardScaler()) 
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [105]:
PREOP_numerical = PREOP_numerical_features_pipeline.fit_transform(X_train_all)

In [106]:
PREOP_numerical.shape, len(PREOP_numerical_col_names)

((4765, 11), 11)

### `PREOP_categorical_features_pipeline`

In [107]:
PREOP_categorical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(PREOP_categorical_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [108]:
PREOP_categorical = PREOP_categorical_features_pipeline.fit_transform(X_train_all)

In [109]:
PREOP_categorical.shape, len(PREOP_categorical_col_names)

((4765, 99), 99)

## Assembling `PREOP_feature_matrix` via `FeatureUnion`

In [110]:
PREOP_feature_matrix = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                     ('PREOP_cat_features', PREOP_categorical_features_pipeline)
                                    ])

In [111]:
type(PREOP_feature_matrix)

sklearn.pipeline.FeatureUnion

### Transforming `X_train_all` to get `X_train_PREOP`

In [112]:
X_train_PREOP = PREOP_feature_matrix.fit_transform(X_train_all)

- validating

In [113]:
X_train_PREOP.shape, y_train.shape

((4765, 110), (4765,))

- putting into a `DataFrame` for easy analysis

In [114]:
PREOP_feature_col_names = PREOP_numerical_col_names + PREOP_categorical_col_names

In [115]:
len(PREOP_feature_col_names), len(PREOP_numerical_col_names), len(PREOP_categorical_col_names)

(110, 11, 99)

In [116]:
X_train_PREOP = pd.DataFrame(X_train_PREOP,
                             columns=PREOP_feature_col_names)

In [117]:
X_train_PREOP.head()

age  heightcm  weightkg      bmi      hct  creatlst  totalbumin  \
0 -0.32301   1.34473  -0.16787 -0.45334 -0.51566   0.17874     0.42969   
1 -1.04541   0.88113  -0.72015 -0.60603  1.60778  -0.39279     1.74170   
2 -0.94221  -2.14592  -1.83965 -0.53043 -0.91155  -0.10703     0.80455   
3  1.43140   2.03559   0.69787 -0.24205  2.32758   0.17874     0.99198   
4 -0.32301  -0.96419  -1.25253 -0.45952 -0.01179  -0.29754     0.42969   

    a1clvl  meldscr     hdef    pasys  surgdt_month_Jan  surgdt_month_Feb  \
0  1.01384  1.19741 -0.32315  3.45717               0.0               0.0   
1 -0.44535 -0.74319  0.19543 -0.17376               0.0               0.0   
2 -0.10201 -0.46012 -1.65665 -0.25820               0.0               0.0   
3 -0.78869  0.27271  0.56585 -0.42708               0.0               0.0   
4 -1.04620 -0.39722  0.19543 -0.25820               0.0               0.0   

   surgdt_month_Mar  surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  \
0               0.0               0.0               1.0               0.0   
1               1.0               0.0               0.0               0.0   
2               1.0               0.0               0.0               0.0   
3               1.0               0.0               0.0               0.0   
4               0.0               0.0               0.0               0.0   

   surgdt_month_Aug  surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  \
0               0.0               0.0               0.0               0.0   
1               0.0               0.0               0.0               0.0   
2               0.0               0.0               0.0               0.0   
3               0.0               0.0               0.0               0.0   
4               0.0               0.0               0.0               0.0   

   surgdt_month_Dec  surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  \
0               0.0                   0.0                    0.0   
1               0.0                   0.0                    0.0   
2               0.0                   0.0                    0.0   
3               0.0                   1.0                    0.0   
4               1.0                   0.0                    1.0   

   surgdt_DayOfWeek_Thurs  surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  \
0                     0.0                   0.0                   0.0   
1                     1.0                   0.0                   0.0   
2                     0.0                   0.0                   0.0   
3                     0.0                   0.0                   0.0   
4                     0.0                   0.0                   0.0   

   surgdt_DayOfWeek_Sun  surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  \
0                   0.0                     1.0                     0.0   
1                   0.0                     0.0                     1.0   
2                   0.0                     0.0                     0.0   
3                   0.0                     0.0                     0.0   
4                   0.0                     0.0                     0.0   

   gender  racecaucasian  raceblack  raceasian  racenativeam  \
0     1.0            1.0        0.0        0.0           0.0   
1     1.0            1.0        0.0        0.0           0.0   
2     0.0            1.0        0.0        0.0           0.0   
3     1.0            1.0        0.0        0.0           0.0   
4     1.0            1.0        0.0        0.0           0.0   

   racnativepacific  ethnicity  diabetes  dyslip  dialysis  hypertn  infendo  \
0               0.0        0.0       1.0     1.0       0.0      1.0      0.0   
1               0.0        0.0       0.0     0.0       0.0      0.0      0.0   
2               0.0        0.0       0.0     1.0       0.0      1.0      0.0   
3               0.0        0.0       0.0     1.0       0.0      1.0      0.0   
4               0.0        0.0       0.0     1.0       0.0      1.0      0.0   

   slpap

### Transforming `X_dev_all` to get `X_dev_PREOP`

In [106]:
#X_dev_PREOP = PREOP_feature_matrix.transform(X_dev_all)

- validating

In [107]:
#X_dev_PREOP.shape, y_dev.shape

((529, 110), (529,))

- putting into a `DataFrame` for easy analysis

In [108]:
# X_dev_PREOP = pd.DataFrame(X_dev_PREOP,
#                            columns=PREOP_feature_col_names)

In [109]:
# X_dev_PREOP.head()

age  heightcm  weightkg      bmi      hct  creatlst  totalbumin  \
0  1.43006   0.68395  -0.03470 -0.23135  0.50953  -0.51096    -0.50284   
1 -0.93023  -0.47993  -0.61014 -0.21612 -1.45135   0.17205     0.05617   
2  0.50647  -0.01979  -1.11612 -0.60286 -0.30751  -0.37062     1.54687   
3  0.30123   0.68395   0.11412 -0.16030  0.90897   0.17205    -0.13017   
4 -1.13547   0.41328  -0.53077 -0.40512 -0.90666  -0.20220    -0.31650   

    a1clvl  meldscr     hdef    pasys  surgdt_month_Jan  surgdt_month_Feb  \
0 -0.44744 -0.29620  0.93653  0.47276               0.0               0.0   
1 -1.12886  1.15767 -1.87899 -0.45297               0.0               0.0   
2  0.23397 -0.74331  1.30699 -0.78959               0.0               0.0   
3  0.23397  0.25721 -2.02718  1.73511               0.0               0.0   
4 -0.78815 -0.61824 -0.32305  1.31433               0.0               0.0   

   surgdt_month_Mar  surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  \
0               0.0               0.0               0.0               1.0   
1               0.0               1.0               0.0               0.0   
2               0.0               0.0               0.0               0.0   
3               0.0               1.0               0.0               0.0   
4               0.0               0.0               0.0               1.0   

   surgdt_month_Aug  surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  \
0               0.0               0.0               0.0               0.0   
1               0.0               0.0               0.0               0.0   
2               0.0               0.0               0.0               1.0   
3               0.0               0.0               0.0               0.0   
4               0.0               0.0               0.0               0.0   

   surgdt_month_Dec  surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  \
0               0.0                   0.0                    0.0   
1               0.0                   1.0                    0.0   
2               0.0                   0.0                    1.0   
3               0.0                   0.0                    0.0   
4               0.0                   1.0                    0.0   

   surgdt_DayOfWeek_Thurs  surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  \
0                     0.0                   1.0                   0.0   
1                     0.0                   0.0                   0.0   
2                     0.0                   0.0                   0.0   
3                     1.0                   0.0                   0.0   
4                     0.0                   0.0                   0.0   

   surgdt_DayOfWeek_Sun  surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  \
0                   0.0                     0.0                     1.0   
1                   0.0                     0.0                     0.0   
2                   0.0                     0.0                     0.0   
3                   0.0                     0.0                     1.0   
4                   0.0                     0.0                     1.0   

   gender  racecaucasian  raceblack  raceasian  racenativeam  \
0     1.0            1.0        0.0        0.0           0.0   
1     1.0            1.0        0.0        0.0           0.0   
2     1.0            1.0        0.0        0.0           0.0   
3     1.0            1.0        0.0        0.0           0.0   
4     1.0            1.0        0.0        0.0           0.0   

   racnativepacific  ethnicity  diabetes  dyslip  dialysis  hypertn  infendo  \
0               0.0        0.0       0.0     1.0       0.0      1.0      0.0   
1               0.0        0.0       1.0     1.0       0.0      1.0      0.0   
2               0.0        0.0       1.0     1.0       0.0      1.0      0.0   
3               0.0        0.0       1.0     1.0       0.0      1.0      0.0   
4               0.0        0.0       0.0     1.0       0.0      1.0      0.0   

   slpap

### Transforming `X_test_all` to get `X_test_PREOP`

In [118]:
X_test_PREOP = PREOP_feature_matrix.transform(X_test_all)

- validating

In [119]:
X_test_PREOP.shape, y_test.shape

((530, 110), (530,))

- putting into a `DataFrame` for easy analysis

In [120]:
X_test_PREOP = pd.DataFrame(X_test_PREOP,
                            columns=PREOP_feature_col_names)

In [121]:
X_test_PREOP.head()

age  heightcm  weightkg      bmi      hct  creatlst  totalbumin  \
0  0.19299   0.39935  -0.24748 -0.26349 -0.01179   0.55976    -0.13260   
1  0.70900  -1.41870  -0.55098  0.11061 -0.01179  -0.10703     0.61712   
2 -1.56141   0.67206  -1.34209 -0.86201  0.25814  -0.39279     0.05483   
3  0.60580   1.34473   0.49388 -0.15831 -0.19174  -0.10703     0.05483   
4  1.53460   0.21754   0.00130 -0.08805  1.60778  -0.10703     1.17941   

    a1clvl  meldscr     hdef    pasys  surgdt_month_Jan  surgdt_month_Feb  \
0 -0.44535  2.46493  0.56585  1.26172               0.0               0.0   
1  0.92801 -0.46012  1.15851 -0.51152               0.0               0.0   
2 -0.53119 -0.34375  0.78810 -0.62974               0.0               0.0   
3 -0.61702 -0.46012  0.26951 -0.00488               0.0               0.0   
4 -0.27368 -0.12044 -1.28624 -2.20033               0.0               0.0   

   surgdt_month_Mar  surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  \
0               0.0               0.0               0.0               0.0   
1               1.0               0.0               0.0               0.0   
2               1.0               0.0               0.0               0.0   
3               0.0               1.0               0.0               0.0   
4               0.0               0.0               0.0               0.0   

   surgdt_month_Aug  surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  \
0               0.0               1.0               0.0               0.0   
1               0.0               0.0               0.0               0.0   
2               0.0               0.0               0.0               0.0   
3               0.0               0.0               0.0               0.0   
4               0.0               0.0               1.0               0.0   

   surgdt_month_Dec  surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  \
0               0.0                   0.0                    1.0   
1               0.0                   0.0                    1.0   
2               0.0                   0.0                    0.0   
3               0.0                   0.0                    0.0   
4               0.0                   0.0                    1.0   

   surgdt_DayOfWeek_Thurs  surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  \
0                     0.0                   0.0                   0.0   
1                     0.0                   0.0                   0.0   
2                     0.0                   0.0                   0.0   
3                     0.0                   1.0                   0.0   
4                     0.0                   0.0                   0.0   

   surgdt_DayOfWeek_Sun  surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  \
0                   0.0                     0.0                     0.0   
1                   0.0                     0.0                     1.0   
2                   0.0                     0.0                     1.0   
3                   0.0                     0.0                     0.0   
4                   0.0                     0.0                     1.0   

   gender  racecaucasian  raceblack  raceasian  racenativeam  \
0     1.0            1.0        0.0        0.0           0.0   
1     0.0            1.0        0.0        0.0           0.0   
2     1.0            1.0        0.0        0.0           0.0   
3     1.0            1.0        0.0        0.0           0.0   
4     1.0            1.0        0.0        0.0           0.0   

   racnativepacific  ethnicity  diabetes  dyslip  dialysis  hypertn  infendo  \
0               0.0        0.0       0.0     1.0       0.0      1.0      0.0   
1               0.0        0.0       1.0     1.0       0.0      1.0      0.0   
2               0.0        0.0       0.0     1.0       0.0      1.0      0.0   
3               0.0        0.0       1.0     1.0       0.0      1.0      0.0   
4               0.0        0.0       0.0     1.0       0.0      1.0      0.0   

   slpap

#### NOTE:
- did not have to put feature matricies into `DataFrames` - could have used directly in pipelines, but wanted to put into `DataFrames` in case wanted to retrieve feature names

# `POSTOP` Feature Matrix Pipelines

### `POSTOP_numerical_features` Pipeline

In [122]:
POSTOP_numerical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(POSTOP_numerical_col_names)), 
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('scaler', StandardScaler()) 
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [123]:
POSTOP_numerical = POSTOP_numerical_features_pipeline.fit_transform(X_train_all)

In [124]:
POSTOP_numerical.shape, len(POSTOP_numerical_col_names)

((4765, 15), 15)

### `POSTOP_categorical_features_pipeline`

In [125]:
POSTOP_categorical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(POSTOP_categorical_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [126]:
POSTOP_categorical = POSTOP_categorical_features_pipeline.fit_transform(X_train_all)

In [127]:
POSTOP_categorical.shape, len(POSTOP_categorical_col_names)

((4765, 96), 96)

## Assembling `POSTOP_feature_matrix` via `FeatureUnion`

In [128]:
POSTOP_feature_matrix = FeatureUnion([('POSTOP_num_features', POSTOP_numerical_features_pipeline),
                                      ('POSTOP_cat_features', POSTOP_categorical_features_pipeline)
                                     ])

In [129]:
type(POSTOP_feature_matrix)

sklearn.pipeline.FeatureUnion

### Transforming `X_train_all` to get `X_train_POSTOP`

In [130]:
X_train_POSTOP = POSTOP_feature_matrix.fit_transform(X_train_all)

- validating

In [131]:
X_train_POSTOP.shape, y_train.shape

((4765, 111), (4765,))

- putting into a `DataFrame` for easy analysis

In [132]:
POSTOP_feature_col_names = POSTOP_numerical_col_names + POSTOP_categorical_col_names

In [133]:
len(POSTOP_feature_col_names), len(POSTOP_numerical_col_names), len(POSTOP_categorical_col_names)

(111, 15, 96)

In [134]:
X_train_POSTOP = pd.DataFrame(X_train_POSTOP,
                              columns=POSTOP_feature_col_names)

In [135]:
X_train_POSTOP.head()

cperftime  cumulsatlft  cumulsatrt   dhcatm  ibdcryou  ibdffpu  ibdplatu  \
0   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   
1   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   
2   -0.00495     -0.07625    -0.08823  0.00954  -0.12586  0.55797  -0.35738   
3   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392   1.33035   
4   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   

   ibdrbcu  lwsthct  lwsttemp  perfustm  postcreat  prerso2lft  prerso2rt  \
0 -0.03049 -0.61060  -0.54267   2.99604    1.43861     0.02756    0.02721   
1 -0.03049  0.72886  -0.54267   0.34339   -0.60120     0.02756    0.02721   
2  0.79022  0.50561   0.15682   0.68165    0.00319     0.02756    0.02721   
3 -1.67189  1.84507  -1.34209  -0.33312   -0.14791     0.02756    0.02721   
4 -0.03049  0.05913   1.05617  -0.70699   -0.45010     0.02756    0.02721   

   xclamptm  canartstaort  canartstfem  canartstoth  canartstax  \
0   2.47082           1.0          0.0          0.0         0.0   
1  -0.11015           1.0          0.0          0.0         0.0   
2  -0.22335           1.0          0.0          0.0         0.0   
3  -0.67615           1.0          0.0          0.0         0.0   
4  -0.47239           1.0          0.0          0.0         0.0   

   cathbasassist  ceroxused  circarr  cofirstind  concalc  cperfutil  IABP  \
0            0.0        1.0      0.0         0.0      0.0        0.0   0.0   
1            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
2            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
3            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
4            0.0        0.0      0.0         0.0      0.0        0.0   0.0   

   ibldprod  imedeaca  imedtran  inoptee  ocarasd  ocarvsd  opocard  oponcard  \
0       0.0       1.0       0.0      1.0      0.0      0.0      1.0       0.0   
1       0.0       1.0       0.0      0.0      0.0      0.0      0.0       0.0   
2       1.0       1.0       0.0      0.0      1.0      0.0      1.0       0.0   
3       1.0       1.0       0.0      0.0      0.0      0.0      0.0       0.0   
4       0.0       1.0       0.0      1.0      0.0      0.0      0.0       0.0   

   unplao  unplav  unplmv  unplvad  vsmvpr  asmtascaa  cotafib  cotarrst  \
0     0.0     0.0     0.0      0.0     1.0        1.0      1.0       0.0   
1     0.0     0.0     0.0      0.0     1.0        1.0      1.0       0.0   
2     0.0     0.0     0.0      0.0     0.0        0.0      0.0       0.0   
3     0.0     0.0     0.0      0.0     0.0        0.0      1.0       0.0   
4     0.0     0.0     0.0      0.0     0.0        1.0      1.0       0.0   

   ecmo  mt30stat  opcab  opvalve  vadproc  vsavpr  vsmv  aortoccl_AoXC  \
0   0.0       1.0    1.0      1.0      0.0     0.0   1.0            1.0   
1   0.0       1.0    1.0      1.0      0.0     0.0   1.0            1.0   
2   0.0       1.0    1.0      1.0      0.0     0.0   1.0            1.0   
3   0.0       1.0    1.0      1.0      0.0     1.0   0.0            1.0   
4   0.0       1.0    1.0      1.0      0.0     1.0   0.0            1.0   

   aortoccl_Balloon  asmtaodx_Normal  asmtaodx_Thickening  \
0               0.0              1.0                  0.0   
1               0.0              1.0                  0.0   
2               0.0              0.0                  0.0   
3               0.0              0.0                  0.0   
4               0.0              1.0                  0.0   

   asmtaodx_atheroma<5mm  asmtaodx_atheroma>5mm  asmtaodx_mobile_plaques  \
0                    0.0                    0.0                      0.0   
1                    0.0                    0.0                      0.0   
2                    0.0                    0.0                      0.0   
3                    0.0                    0.0                      0.0   
4                    0.0                    0.0    

### Transforming `X_dev_all` to get `X_dev_POSTOP`

In [128]:
# X_dev_POSTOP = POSTOP_feature_matrix.transform(X_dev_all)

- validating

In [129]:
# X_dev_POSTOP.shape, y_dev.shape

((529, 118), (529,))

- putting into a `DataFrame` for easy analysis

In [130]:
# X_dev_POSTOP = pd.DataFrame(X_dev_POSTOP,
#                             columns=POSTOP_feature_col_names)

In [131]:
# X_dev_POSTOP.head()

cperftime  cumulsatlft  cumulsatrt   dhcatm  ibdcryou  ibdffpu  ibdplatu  \
0   -0.00525     -0.07508    -0.09078  0.01012  -0.12913  1.40194   0.47704   
1   -0.00525     -0.07508    -0.09078  0.01012  -0.12913 -0.33399  -0.35729   
2   -0.00525     -0.07508    -0.09078  0.01012  -0.12913 -0.33399   1.31138   
3   -0.00525     -0.07508    -0.09078  0.01012  -0.12913  1.40194   0.47704   
4   -0.00525     -0.07508    -0.09078  0.01012  -0.12913 -0.33399  -0.35729   

   ibdrbcu  lwsthct  lwsttemp  perfustm  postcreat  prerso2lft  prerso2rt  \
0 -1.65040  0.51881   1.19790   0.59879   -0.37193     0.02817    0.03293   
1 -0.84738 -0.59524  -0.33417  -1.53804    0.07449     0.02817    0.03293   
2 -0.04436 -0.90718  -0.63069  -0.88463   -0.52073     0.02817    0.03293   
3 -1.65040  1.05356   0.16005  -0.39016    0.00009     0.02817    0.03293   
4 -0.04436 -0.37243  -0.18590   1.11092   -0.14872     0.02817    0.03293   

   xclamptm  canartstaort  canartstfem  canartstoth  canartstax  \
0  -1.08276           1.0          0.0          0.0         0.0   
1  -1.48774           1.0          0.0          0.0         0.0   
2  -0.74528           1.0          0.0          0.0         0.0   
3  -0.34030           1.0          0.0          0.0         0.0   
4   1.27961           1.0          0.0          0.0         0.0   

   cathbasassist  ceroxused  circarr  cofirstind  concalc  cperfutil  IABP  \
0            0.0        0.0      0.0         0.0      0.0        0.0   1.0   
1            0.0        1.0      0.0         0.0      0.0        0.0   0.0   
2            0.0        1.0      0.0         0.0      0.0        0.0   0.0   
3            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
4            0.0        0.0      0.0         0.0      0.0        0.0   0.0   

   ibldprod  imedeaca  imedtran  inoptee  mtopd  ocarasd  ocarvsd  opocard  \
0       1.0       1.0       0.0      1.0    0.0      0.0      0.0      1.0   
1       1.0       1.0       0.0      1.0    0.0      0.0      0.0      1.0   
2       1.0       1.0       0.0      1.0    0.0      0.0      0.0      0.0   
3       1.0       1.0       0.0      1.0    0.0      0.0      0.0      0.0   
4       0.0       1.0       0.0      1.0    0.0      0.0      0.0      1.0   

   oponcard  unplao  unplav  unplmv  unplvad  vsmvpr  asmtascaa  cotafib  \
0       0.0     0.0     0.0     0.0      0.0     0.0        1.0      0.0   
1       0.0     0.0     0.0     0.0      0.0     0.0        1.0      0.0   
2       0.0     0.0     0.0     0.0      0.0     0.0        1.0      1.0   
3       0.0     0.0     0.0     0.0      0.0     0.0        1.0      0.0   
4       0.0     0.0     0.0     0.0      0.0     0.0        1.0      0.0   

   cotarrst  ecmo  mt30stat  opcab  opvalve  vadproc  vsavpr  vsmv  \
0       0.0   0.0       1.0    1.0      1.0      0.0     1.0   0.0   
1       0.0   0.0       1.0    0.0      1.0      0.0     1.0   0.0   
2       0.0   0.0       1.0    1.0      1.0      0.0     1.0   0.0   
3       0.0   0.0       1.0    1.0      1.0      0.0     1.0   0.0   
4       0.0   0.0       1.0    1.0      1.0      0.0     0.0   1.0   

   aortoccl_AoXC  aortoccl_Balloon  asmtaodx_Normal  asmtaodx_Thickening  \
0            1.0               0.0              0.0                  0.0   
1            1.0               0.0              1.0                  0.0   
2            1.0               0.0              1.0                  0.0   
3            1.0               0.0              0.0                  1.0   
4            1.0               0.0              0.0                  0.0   

   asmtaodx_atheroma<5mm  asmtaodx_atheroma>5mm  asmtaodx_mobile_plaques  \
0                    1.0                    0.0                      0.0   
1                    0.0                    0.0                      0.0   
2                    0.0                    0.0                      0.0   
3                    0.0                    0.0                      0.0   
4        

### Transforming `X_test_all` to get `X_test_POSTOP`

In [136]:
X_test_POSTOP = POSTOP_feature_matrix.transform(X_test_all)

- validating

In [137]:
X_test_POSTOP.shape, y_test.shape

((530, 111), (530,))

- putting into a `DataFrame` for easy analysis

In [138]:
X_test_POSTOP = pd.DataFrame(X_test_POSTOP,
                             columns=POSTOP_feature_col_names)

In [139]:
X_test_POSTOP.head()

cperftime  cumulsatlft  cumulsatrt   dhcatm  ibdcryou  ibdffpu  ibdplatu  \
0   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   
1   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392   1.33035   
2   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   
3   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   
4   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   

   ibdrbcu  lwsthct  lwsttemp  perfustm  postcreat  prerso2lft  prerso2rt  \
0 -0.03049 -0.14179  -2.84101  -0.97403    0.22983     0.02756    0.02721   
1  1.61092 -1.72681  -1.54195   0.55703   -0.67675     0.02756    0.02721   
2 -0.03049 -0.16411  -0.34282  -0.38653   -0.52565     0.02756    0.02721   
3 -0.03049  2.51480   2.25531  -0.61797   -0.37456     0.02756    0.02721   
4  1.61092  0.28237   0.15682  -0.35093   -0.29901     0.02756    0.02721   

   xclamptm  canartstaort  canartstfem  canartstoth  canartstax  \
0  -0.74407           1.0          0.0          0.0         0.0   
1   0.16153           1.0          0.0          0.0         0.0   
2  -0.51767           1.0          0.0          0.0         0.0   
3  -0.26863           1.0          0.0          0.0         0.0   
4  -0.54031           1.0          0.0          0.0         0.0   

   cathbasassist  ceroxused  circarr  cofirstind  concalc  cperfutil  IABP  \
0            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
1            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
2            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
3            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
4            0.0        0.0      0.0         0.0      0.0        0.0   0.0   

   ibldprod  imedeaca  imedtran  inoptee  ocarasd  ocarvsd  opocard  oponcard  \
0       0.0       1.0       0.0      1.0      0.0      0.0      0.0       0.0   
1       1.0       1.0       0.0      1.0      0.0      0.0      0.0       0.0   
2       0.0       1.0       0.0      0.0      0.0      0.0      0.0       0.0   
3       0.0       1.0       0.0      0.0      0.0      0.0      0.0       0.0   
4       1.0       1.0       0.0      0.0      0.0      0.0      0.0       0.0   

   unplao  unplav  unplmv  unplvad  vsmvpr  asmtascaa  cotafib  cotarrst  \
0     0.0     0.0     0.0      0.0     1.0        0.0      0.0       0.0   
1     0.0     0.0     0.0      0.0     0.0        1.0      1.0       0.0   
2     0.0     0.0     0.0      0.0     0.0        0.0      0.0       0.0   
3     0.0     0.0     0.0      0.0     0.0        1.0      0.0       0.0   
4     0.0     0.0     0.0      0.0     0.0        0.0      1.0       0.0   

   ecmo  mt30stat  opcab  opvalve  vadproc  vsavpr  vsmv  aortoccl_AoXC  \
0   0.0       1.0    1.0      1.0      0.0     0.0   1.0            1.0   
1   0.0       1.0    1.0      1.0      0.0     1.0   0.0            1.0   
2   0.0       1.0    1.0      1.0      0.0     1.0   0.0            1.0   
3   0.0       1.0    1.0      1.0      0.0     1.0   0.0            1.0   
4   0.0       1.0    1.0      1.0      0.0     1.0   0.0            1.0   

   aortoccl_Balloon  asmtaodx_Normal  asmtaodx_Thickening  \
0               0.0              0.0                  0.0   
1               0.0              1.0                  0.0   
2               0.0              0.0                  0.0   
3               0.0              0.0                  1.0   
4               0.0              0.0                  0.0   

   asmtaodx_atheroma<5mm  asmtaodx_atheroma>5mm  asmtaodx_mobile_plaques  \
0                    0.0                    0.0                      0.0   
1                    0.0                    0.0                      0.0   
2                    0.0                    0.0                      0.0   
3                    0.0                    0.0                      0.0   
4                    0.0                    0.0    

## Assembling `PREOP + POSTOP` Combined Feature Matrix

#### -  Using `FeatureUnion | Pipeline` to Create Combined `PREOP + POSTOP` Feature Matrix

In [140]:
PREPOST_feature_matrix = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                       ('PREOP_cat_features', PREOP_categorical_features_pipeline),
                                       ('POSTOP_num_features', POSTOP_numerical_features_pipeline),
                                       ('POSTOP_cat_features', POSTOP_categorical_features_pipeline)
                                      ])

- can use the code pattern above to create `X_train_PREPOST`, `X_dev_PREPOST`, `X_test_PREPOST`

In [141]:
PREPOST_feature_col_names = PREOP_feature_col_names + POSTOP_feature_col_names

In [142]:
len(PREPOST_feature_col_names), len(PREOP_feature_col_names), len(POSTOP_feature_col_names)

(221, 110, 111)

### Transforming `X_train_all` to get `X_train_PREPOST`

In [143]:
X_train_PREPOST = PREPOST_feature_matrix.fit_transform(X_train_all)

- validating

In [144]:
X_train_PREPOST.shape, y_train.shape

((4765, 221), (4765,))

- putting into a `DataFrame` for easy analysis

In [145]:
X_train_PREPOST = pd.DataFrame(X_train_PREPOST,
                               columns=PREPOST_feature_col_names)

In [146]:
X_train_PREPOST.head()

age  heightcm  weightkg      bmi      hct  creatlst  totalbumin  \
0 -0.32301   1.34473  -0.16787 -0.45334 -0.51566   0.17874     0.42969   
1 -1.04541   0.88113  -0.72015 -0.60603  1.60778  -0.39279     1.74170   
2 -0.94221  -2.14592  -1.83965 -0.53043 -0.91155  -0.10703     0.80455   
3  1.43140   2.03559   0.69787 -0.24205  2.32758   0.17874     0.99198   
4 -0.32301  -0.96419  -1.25253 -0.45952 -0.01179  -0.29754     0.42969   

    a1clvl  meldscr     hdef    pasys  surgdt_month_Jan  surgdt_month_Feb  \
0  1.01384  1.19741 -0.32315  3.45717               0.0               0.0   
1 -0.44535 -0.74319  0.19543 -0.17376               0.0               0.0   
2 -0.10201 -0.46012 -1.65665 -0.25820               0.0               0.0   
3 -0.78869  0.27271  0.56585 -0.42708               0.0               0.0   
4 -1.04620 -0.39722  0.19543 -0.25820               0.0               0.0   

   surgdt_month_Mar  surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  \
0               0.0               0.0               1.0               0.0   
1               1.0               0.0               0.0               0.0   
2               1.0               0.0               0.0               0.0   
3               1.0               0.0               0.0               0.0   
4               0.0               0.0               0.0               0.0   

   surgdt_month_Aug  surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  \
0               0.0               0.0               0.0               0.0   
1               0.0               0.0               0.0               0.0   
2               0.0               0.0               0.0               0.0   
3               0.0               0.0               0.0               0.0   
4               0.0               0.0               0.0               0.0   

   surgdt_month_Dec  surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  \
0               0.0                   0.0                    0.0   
1               0.0                   0.0                    0.0   
2               0.0                   0.0                    0.0   
3               0.0                   1.0                    0.0   
4               1.0                   0.0                    1.0   

   surgdt_DayOfWeek_Thurs  surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  \
0                     0.0                   0.0                   0.0   
1                     1.0                   0.0                   0.0   
2                     0.0                   0.0                   0.0   
3                     0.0                   0.0                   0.0   
4                     0.0                   0.0                   0.0   

   surgdt_DayOfWeek_Sun  surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  \
0                   0.0                     1.0                     0.0   
1                   0.0                     0.0                     1.0   
2                   0.0                     0.0                     0.0   
3                   0.0                     0.0                     0.0   
4                   0.0                     0.0                     0.0   

   gender  racecaucasian  raceblack  raceasian  racenativeam  \
0     1.0            1.0        0.0        0.0           0.0   
1     1.0            1.0        0.0        0.0           0.0   
2     0.0            1.0        0.0        0.0           0.0   
3     1.0            1.0        0.0        0.0           0.0   
4     1.0            1.0        0.0        0.0           0.0   

   racnativepacific  ethnicity  diabetes  dyslip  dialysis  hypertn  infendo  \
0               0.0        0.0       1.0     1.0       0.0      1.0      0.0   
1               0.0        0.0       0.0     0.0       0.0      0.0      0.0   
2               0.0        0.0       0.0     1.0       0.0      1.0      0.0   
3               0.0        0.0       0.0     1.0       0.0      1.0      0.0   
4               0.0        0.0       0.0     1.0       0.0      1.0      0.0   

   slpap

### Transforming `X_dev_all` to get `X_dev_PREPOST`

In [143]:
# X_dev_PREPOST = PREPOST_feature_matrix.transform(X_dev_all)

- validating

In [144]:
# X_dev_PREPOST.shape, y_dev.shape

((529, 228), (529,))

- putting into a `DataFrame` for easy analysis

In [145]:
# X_dev_PREPOST = pd.DataFrame(X_dev_PREPOST,
#                              columns=PREPOST_feature_col_names)

In [146]:
# X_dev_PREPOST.head()

age  heightcm  weightkg      bmi      hct  creatlst  totalbumin  \
0  1.43006   0.68395  -0.03470 -0.23135  0.50953  -0.51096    -0.50284   
1 -0.93023  -0.47993  -0.61014 -0.21612 -1.45135   0.17205     0.05617   
2  0.50647  -0.01979  -1.11612 -0.60286 -0.30751  -0.37062     1.54687   
3  0.30123   0.68395   0.11412 -0.16030  0.90897   0.17205    -0.13017   
4 -1.13547   0.41328  -0.53077 -0.40512 -0.90666  -0.20220    -0.31650   

    a1clvl  meldscr     hdef    pasys  surgdt_month_Jan  surgdt_month_Feb  \
0 -0.44744 -0.29620  0.93653  0.47276               0.0               0.0   
1 -1.12886  1.15767 -1.87899 -0.45297               0.0               0.0   
2  0.23397 -0.74331  1.30699 -0.78959               0.0               0.0   
3  0.23397  0.25721 -2.02718  1.73511               0.0               0.0   
4 -0.78815 -0.61824 -0.32305  1.31433               0.0               0.0   

   surgdt_month_Mar  surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  \
0               0.0               0.0               0.0               1.0   
1               0.0               1.0               0.0               0.0   
2               0.0               0.0               0.0               0.0   
3               0.0               1.0               0.0               0.0   
4               0.0               0.0               0.0               1.0   

   surgdt_month_Aug  surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  \
0               0.0               0.0               0.0               0.0   
1               0.0               0.0               0.0               0.0   
2               0.0               0.0               0.0               1.0   
3               0.0               0.0               0.0               0.0   
4               0.0               0.0               0.0               0.0   

   surgdt_month_Dec  surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  \
0               0.0                   0.0                    0.0   
1               0.0                   1.0                    0.0   
2               0.0                   0.0                    1.0   
3               0.0                   0.0                    0.0   
4               0.0                   1.0                    0.0   

   surgdt_DayOfWeek_Thurs  surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  \
0                     0.0                   1.0                   0.0   
1                     0.0                   0.0                   0.0   
2                     0.0                   0.0                   0.0   
3                     1.0                   0.0                   0.0   
4                     0.0                   0.0                   0.0   

   surgdt_DayOfWeek_Sun  surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  \
0                   0.0                     0.0                     1.0   
1                   0.0                     0.0                     0.0   
2                   0.0                     0.0                     0.0   
3                   0.0                     0.0                     1.0   
4                   0.0                     0.0                     1.0   

   gender  racecaucasian  raceblack  raceasian  racenativeam  \
0     1.0            1.0        0.0        0.0           0.0   
1     1.0            1.0        0.0        0.0           0.0   
2     1.0            1.0        0.0        0.0           0.0   
3     1.0            1.0        0.0        0.0           0.0   
4     1.0            1.0        0.0        0.0           0.0   

   racnativepacific  ethnicity  diabetes  dyslip  dialysis  hypertn  infendo  \
0               0.0        0.0       0.0     1.0       0.0      1.0      0.0   
1               0.0        0.0       1.0     1.0       0.0      1.0      0.0   
2               0.0        0.0       1.0     1.0       0.0      1.0      0.0   
3               0.0        0.0       1.0     1.0       0.0      1.0      0.0   
4               0.0        0.0       0.0     1.0       0.0      1.0      0.0   

   slpap

### Transforming `X_test_all` to get `X_test_PREPOST`

In [147]:
X_test_PREPOST = PREPOST_feature_matrix.transform(X_test_all)

- validating

In [148]:
X_test_PREPOST.shape, y_test.shape

((530, 221), (530,))

- putting into a `DataFrame` for easy analysis

In [149]:
X_test_PREPOST = pd.DataFrame(X_test_PREPOST,
                              columns=PREPOST_feature_col_names)

In [150]:
X_test_PREPOST.head()

age  heightcm  weightkg      bmi      hct  creatlst  totalbumin  \
0  0.19299   0.39935  -0.24748 -0.26349 -0.01179   0.55976    -0.13260   
1  0.70900  -1.41870  -0.55098  0.11061 -0.01179  -0.10703     0.61712   
2 -1.56141   0.67206  -1.34209 -0.86201  0.25814  -0.39279     0.05483   
3  0.60580   1.34473   0.49388 -0.15831 -0.19174  -0.10703     0.05483   
4  1.53460   0.21754   0.00130 -0.08805  1.60778  -0.10703     1.17941   

    a1clvl  meldscr     hdef    pasys  surgdt_month_Jan  surgdt_month_Feb  \
0 -0.44535  2.46493  0.56585  1.26172               0.0               0.0   
1  0.92801 -0.46012  1.15851 -0.51152               0.0               0.0   
2 -0.53119 -0.34375  0.78810 -0.62974               0.0               0.0   
3 -0.61702 -0.46012  0.26951 -0.00488               0.0               0.0   
4 -0.27368 -0.12044 -1.28624 -2.20033               0.0               0.0   

   surgdt_month_Mar  surgdt_month_Apr  surgdt_month_May  surgdt_month_Jul  \
0               0.0               0.0               0.0               0.0   
1               1.0               0.0               0.0               0.0   
2               1.0               0.0               0.0               0.0   
3               0.0               1.0               0.0               0.0   
4               0.0               0.0               0.0               0.0   

   surgdt_month_Aug  surgdt_month_Sep  surgdt_month_Oct  surgdt_month_Nov  \
0               0.0               1.0               0.0               0.0   
1               0.0               0.0               0.0               0.0   
2               0.0               0.0               0.0               0.0   
3               0.0               0.0               0.0               0.0   
4               0.0               0.0               1.0               0.0   

   surgdt_month_Dec  surgdt_DayOfWeek_Mon  surgdt_DayOfWeek_Tues  \
0               0.0                   0.0                    1.0   
1               0.0                   0.0                    1.0   
2               0.0                   0.0                    0.0   
3               0.0                   0.0                    0.0   
4               0.0                   0.0                    1.0   

   surgdt_DayOfWeek_Thurs  surgdt_DayOfWeek_Fri  surgdt_DayOfWeek_Sat  \
0                     0.0                   0.0                   0.0   
1                     0.0                   0.0                   0.0   
2                     0.0                   0.0                   0.0   
3                     0.0                   1.0                   0.0   
4                     0.0                   0.0                   0.0   

   surgdt_DayOfWeek_Sun  surgdt_PartOfMonth_Beg  surgdt_PartOfMonth_End  \
0                   0.0                     0.0                     0.0   
1                   0.0                     0.0                     1.0   
2                   0.0                     0.0                     1.0   
3                   0.0                     0.0                     0.0   
4                   0.0                     0.0                     1.0   

   gender  racecaucasian  raceblack  raceasian  racenativeam  \
0     1.0            1.0        0.0        0.0           0.0   
1     0.0            1.0        0.0        0.0           0.0   
2     1.0            1.0        0.0        0.0           0.0   
3     1.0            1.0        0.0        0.0           0.0   
4     1.0            1.0        0.0        0.0           0.0   

   racnativepacific  ethnicity  diabetes  dyslip  dialysis  hypertn  infendo  \
0               0.0        0.0       0.0     1.0       0.0      1.0      0.0   
1               0.0        0.0       1.0     1.0       0.0      1.0      0.0   
2               0.0        0.0       0.0     1.0       0.0      1.0      0.0   
3               0.0        0.0       1.0     1.0       0.0      1.0      0.0   
4               0.0        0.0       0.0     1.0       0.0      1.0      0.0   

   slpap

## Modeling Scenarios Using `PREOP` and `POSTOP` Datasets

### - Scenario A: `Kitchen Sink` - Modeling the Combined `PREOP` and `POSTOP` Dataset

- code belows shows feature matrix assembly using `Pipeline` - could have used the already named `feature matricies` created above: `X_train_PREPOST`, `X_dev_PREPOST` and `X_test_PREPOST` in conjunction with their respective `y` vectors

#### `LogisticRegression()` with `GridSearchCV`

- instantiating `Pipeline`

In [151]:
PREPOST_modelA_pipe = Pipeline(steps=[
    ('get_PREPOST_feature_matrix', PREPOST_feature_matrix),
    ('Classifier', LogisticRegression())
    ])

- defining a parameter grid for `GridSearchCV`

In [152]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.001, 0.01, 1.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

- defining `scoring_metrics`

In [153]:
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

- create `GridSearchCV` object

In [154]:
gs = GridSearchCV(PREPOST_modelA_pipe, #pipline
                  param_grid=parameter_grid, #param_grid
                  scoring=scoring_metrics, #loss functions
                  cv=5, #number of folds, default=`StratifiedKFold`
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  return_train_score=True,
                  n_jobs=-1,
                  verbose=5)

- fitting `GridSearchCV`

In [155]:
PREPOST_modelA = gs.fit(X_train_all, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:    9.7s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   14.5s finished


- examining results

In [156]:
gs_results = pd.DataFrame(PREPOST_modelA.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                  ascending=False)

In [157]:
display_cols = ['param_Classifier__C',
                'param_Classifier__penalty',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [158]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [159]:
gs_results_summary.rename(columns={'param_Classifier__C': 'C',
                                   'param_Classifier__penalty': 'penalty'},
                          inplace=True)

In [160]:
gs_results_summary

C penalty  mean_test_roc_auc  mean_train_roc_auc  mean_test_f1  \
0   0.01      l1            0.64703             0.71509       0.08662   
1      1      l1            0.59865             0.90507       0.08237   
2  0.001      l1            0.50000             0.50000       0.00000   

   mean_test_precision  mean_test_recall  
0              0.04690           0.56628  
1              0.04681           0.34284  
2              0.00000           0.00000

#### Key Takeaways
- `LogisticRegression()` with `C=0.01` beats `STS`

## Scenario Ab - Just Preop Data Logistic

### Step 3b: Model and Evaluate Stacked Model Using `RandomForestClassifier`

In [161]:
PRE_feature_matrix = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                       ('PREOP_cat_features', PREOP_categorical_features_pipeline)
                                      ])

In [162]:
PRE_modelA_pipe = Pipeline(steps=[
    ('get_PRE_feature_matrix', PRE_feature_matrix),
    ('Classifier', LogisticRegression())
    ])

In [163]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.0275, 0.01, 1.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

In [164]:
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

In [165]:
gs = GridSearchCV(PRE_modelA_pipe, #pipline
                  param_grid=parameter_grid, #param_grid
                  scoring=scoring_metrics, #loss functions
                  cv=5, #number of folds, default=`StratifiedKFold`
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  return_train_score=True,
                  n_jobs=-1,
                  verbose=5)

In [166]:
PRE_modelA = gs.fit(X_train_all, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:    3.1s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    4.0s finished


In [167]:
gs_results = pd.DataFrame(PRE_modelA.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                  ascending=False)
display_cols = ['param_Classifier__C',
                'param_Classifier__penalty',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)
gs_results_summary.rename(columns={'param_Classifier__C': 'C',
                                   'param_Classifier__penalty': 'penalty'},
                          inplace=True)

In [168]:
gs_results_summary

C penalty  mean_test_roc_auc  mean_train_roc_auc  mean_test_f1  \
0  0.0275      l1            0.64406             0.75306       0.08834   
1    0.01      l1            0.61086             0.67485       0.08006   
2       1      l1            0.58630             0.83984       0.07453   

   mean_test_precision  mean_test_recall  
0              0.04824           0.52464  
1              0.04289           0.60197  
2              0.04128           0.38520

### - Scenario B: `Model Stacking` - Incorporate Predictions from `PREOP` Training with `POSTOP` Data
- this time, we will make use of pre-saved `X_train_PREOP` and `X_train_POSTOP` feature matricies created above via `Pipeline` 

In [169]:
X_train_PREOP.shape, X_train_POSTOP.shape, y_train.shape

((4765, 110), (4765, 111), (4765,))

#### Step 1.  Generate vector of probabilities of stroke for using `PREOP` features and cross validated training set
- `LogisticRegression` hyperparameters were tuned in a separate notebook

In [170]:
lr_predict_probs = cross_val_predict(LogisticRegression(penalty='l1',
                                                        C=0.0275,
                                                        class_weight='balanced',
                                                        random_state=0,
                                                        solver='liblinear'),
                                     X_train_PREOP,
                                     y_train,
                                     cv=5,
                                     method='predict_proba',
                                     n_jobs=-1,
                                     verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


- `cross_val_predict` returns an `numpy.ndarray` with `shape` `(n, 2)` as it returns a probability per class `(0, 1)`
- since we are interested in the `1` class, need to carve our the second column of the `numpy.ndarray`
- add `[:,1]`

- examining `lr_predict_probs`

In [171]:
type(lr_predict_probs)

numpy.ndarray

In [172]:
lr_predict_probs.shape

(4765, 2)

In [173]:
lr_predict_probs[0:5]

array([[0.60526477, 0.39473523],
       [0.57780787, 0.42219213],
       [0.34973439, 0.65026561],
       [0.81073837, 0.18926163],
       [0.59292551, 0.40707449]])

In [174]:
lr_predict_probs[0:5, 1]

array([0.39473523, 0.42219213, 0.65026561, 0.18926163, 0.40707449])

### Step 1a.  Generate additional vector of probabilities of stroke for using `PREOP` features and CV training set
- Not limited to stacking one `PREOP` model
- Running `RandomForestClassifier` `PREOP` model 
- hyperparameters were tuned in a separate notebook - using AP's highest scoring model
- going to assemble `X_train_PREOP_TREE` using `Pipelines`
- can utilize existing `PREOP_numerical_features_pipeline`
- need to define `PREOP_categorial_TREE_features_pipeline`
- use`FeatureUnion` to combine to create `PREOP` feature matrix usable for `RandomForestClassifier`

- creating `Pipeline` to assemble `PREOP_categorical_TREE_features_pipeline`

In [175]:
PREOP_categorical_TREE_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(PREOP_categorical_TREE_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

In [176]:
PREOP_feature_matrix_TREE = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                          ('PREOP_cat_tree_features', PREOP_categorical_TREE_features_pipeline)
                                         ])

- combining with `PREOP_numerical_features_pipeline` defined above to assemble `PREOP_feature_matrix_TREE`
- if plan is to assemble `feature_matrix` but not use in a modeling pipeline as is done below, then can assign to a variable name like `X_train_PREOP_TREE` -- did this above for `X_train_PREOP` 

#### Modeling `PREOP_dataset_TREE` using `RandomForestClassifier()`

- creating `PREOP_RF_CLF_pipe`

In [177]:
PREOP_RF_clf_pipe = Pipeline(steps=[
    ('get_PREOP_TREE_feature_matrix', PREOP_feature_matrix_TREE),
    ('Classifier', RandomForestClassifier(n_estimators=315,
                                          max_features=0.1,
                                          max_depth=None,
                                          min_samples_split=50,
                                          class_weight='balanced',
                                          random_state=0))])

- in this scenario - assuming already have tuned the hyperparamenters
- alternatively, could have run a `GridSearchCV` first to tune the hyperparameters and then use the `best_estimator_` to get estimated probabilities of stroke with the code below

- running `RandomForestClassifer`

In [178]:
rf_predict_probs = cross_val_predict(PREOP_RF_clf_pipe,
                                     X_train_all,
                                     y_train,
                                     cv=5,
                                     method='predict_proba',
                                     n_jobs=-1,
                                     verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    4.0s remaining:    6.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.6s finished


- examining `rf_predict_probs`

In [179]:
rf_predict_probs.shape

(4765, 2)

In [180]:
rf_predict_probs[0:5, 1]

array([0.29987459, 0.08666775, 0.28120929, 0.09920073, 0.12942512])

### Step 2: Incorporate vector of predicted probabilities into `X_train_POSTOP`
- adding `lr_predict_probs`
- adding `rf_predict_probs`
- adding `STS` model predicted probability using `predstro_train`

In [181]:
X_train_POSTOP_STACKED = X_train_POSTOP.copy()

In [182]:
X_train_POSTOP_STACKED.shape, X_train_POSTOP.shape

((4765, 111), (4765, 111))

- adding vectors of `predict_probas` from `PREOP` models and `STS` models to `POSTOP` feature matrix

In [183]:
X_train_POSTOP_STACKED['LR_PREOP_PROBA'] = lr_predict_probs[:, 1]
X_train_POSTOP_STACKED['RF_PREOP_PROBA'] = rf_predict_probs[:, 1]
#X_train_POSTOP_STACKED['STS_predstro'] = predstro_train

In [184]:
X_train_POSTOP_STACKED.head()

cperftime  cumulsatlft  cumulsatrt   dhcatm  ibdcryou  ibdffpu  ibdplatu  \
0   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   
1   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   
2   -0.00495     -0.07625    -0.08823  0.00954  -0.12586  0.55797  -0.35738   
3   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392   1.33035   
4   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   

   ibdrbcu  lwsthct  lwsttemp  perfustm  postcreat  prerso2lft  prerso2rt  \
0 -0.03049 -0.61060  -0.54267   2.99604    1.43861     0.02756    0.02721   
1 -0.03049  0.72886  -0.54267   0.34339   -0.60120     0.02756    0.02721   
2  0.79022  0.50561   0.15682   0.68165    0.00319     0.02756    0.02721   
3 -1.67189  1.84507  -1.34209  -0.33312   -0.14791     0.02756    0.02721   
4 -0.03049  0.05913   1.05617  -0.70699   -0.45010     0.02756    0.02721   

   xclamptm  canartstaort  canartstfem  canartstoth  canartstax  \
0   2.47082           1.0          0.0          0.0         0.0   
1  -0.11015           1.0          0.0          0.0         0.0   
2  -0.22335           1.0          0.0          0.0         0.0   
3  -0.67615           1.0          0.0          0.0         0.0   
4  -0.47239           1.0          0.0          0.0         0.0   

   cathbasassist  ceroxused  circarr  cofirstind  concalc  cperfutil  IABP  \
0            0.0        1.0      0.0         0.0      0.0        0.0   0.0   
1            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
2            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
3            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
4            0.0        0.0      0.0         0.0      0.0        0.0   0.0   

   ibldprod  imedeaca  imedtran  inoptee  ocarasd  ocarvsd  opocard  oponcard  \
0       0.0       1.0       0.0      1.0      0.0      0.0      1.0       0.0   
1       0.0       1.0       0.0      0.0      0.0      0.0      0.0       0.0   
2       1.0       1.0       0.0      0.0      1.0      0.0      1.0       0.0   
3       1.0       1.0       0.0      0.0      0.0      0.0      0.0       0.0   
4       0.0       1.0       0.0      1.0      0.0      0.0      0.0       0.0   

   unplao  unplav  unplmv  unplvad  vsmvpr  asmtascaa  cotafib  cotarrst  \
0     0.0     0.0     0.0      0.0     1.0        1.0      1.0       0.0   
1     0.0     0.0     0.0      0.0     1.0        1.0      1.0       0.0   
2     0.0     0.0     0.0      0.0     0.0        0.0      0.0       0.0   
3     0.0     0.0     0.0      0.0     0.0        0.0      1.0       0.0   
4     0.0     0.0     0.0      0.0     0.0        1.0      1.0       0.0   

   ecmo  mt30stat  opcab  opvalve  vadproc  vsavpr  vsmv  aortoccl_AoXC  \
0   0.0       1.0    1.0      1.0      0.0     0.0   1.0            1.0   
1   0.0       1.0    1.0      1.0      0.0     0.0   1.0            1.0   
2   0.0       1.0    1.0      1.0      0.0     0.0   1.0            1.0   
3   0.0       1.0    1.0      1.0      0.0     1.0   0.0            1.0   
4   0.0       1.0    1.0      1.0      0.0     1.0   0.0            1.0   

   aortoccl_Balloon  asmtaodx_Normal  asmtaodx_Thickening  \
0               0.0              1.0                  0.0   
1               0.0              1.0                  0.0   
2               0.0              0.0                  0.0   
3               0.0              0.0                  0.0   
4               0.0              1.0                  0.0   

   asmtaodx_atheroma<5mm  asmtaodx_atheroma>5mm  asmtaodx_mobile_plaques  \
0                    0.0                    0.0                      0.0   
1                    0.0                    0.0                      0.0   
2                    0.0                    0.0                      0.0   
3                    0.0                    0.0                      0.0   
4                    0.0                    0.0    

In [185]:
X_train_POSTOP_STACKED.shape

(4765, 113)

### Step 3a: Model and Evaluate `X_train_POSTOP_STACKED` Using `LogisticRegression()`

- instantiating a `LogisticRegression()` object

In [186]:
log_reg_clf = LogisticRegression()

- will use scoring metrics defined above in previous `GridSearchCV`
- defining hyperparameter grid

In [187]:
hyperparameters = {'penalty': ['l1'],
                   'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
                   'class_weight': ['balanced'],
                   'solver': ['liblinear'],
                   'random_state': [0]
                  }

- create `GridSearchCV` object

In [188]:
gs_stacked = GridSearchCV(log_reg_clf, #classifier
                          param_grid=hyperparameters, #param_grid
                          scoring=scoring_metrics, #loss functions
                          cv=5, #number of folds, default=`StratifiedKFold`
                          refit='roc_auc', # best_estimator_ will be based on this scoring metric
                          return_train_score=True,
                          n_jobs=-1,
                          verbose=5)

- fitting `GridSearchCV`

In [189]:
PREPOST_modelB = gs_stacked.fit(X_train_POSTOP_STACKED, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   30.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   30.5s finished


- examining results

In [190]:
gs_results = pd.DataFrame(PREPOST_modelB.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                      ascending=False)

In [191]:
display_cols = ['param_C',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [192]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [193]:
gs_results_summary

param_C  mean_test_roc_auc  mean_train_roc_auc  mean_test_f1  \
0     0.1            0.63462             0.75412       0.08953   
1       1            0.60398             0.78702       0.08795   
2    0.01            0.58618             0.65451       0.06652   
3      10            0.58510             0.79311       0.08387   
4  0.0001            0.50000             0.50000       0.00000   
5   0.001            0.50000             0.50000       0.00000   

   mean_test_precision  mean_test_recall  
0              0.04937           0.48230  
1              0.04857           0.46826  
2              0.03626           0.40543  
3              0.04630           0.44708  
4              0.00000           0.00000  
5              0.00000           0.00000

### Key Takeaways
- similar to `Modeling Scenario A` achieved better results than `STS` with the `POSTOP` features
- results were slightly below that of `Modeling Scenario A` on a `mean_test_roc_auc` basis
- extend this stacking concept to add additional vectors of probability of stroke from other models `RandomForest` and `STS`
- adding the `RandomForest` predicted probabilities improved results slightly, but the addition of `STS` on top of that did not improve the `mean_test_roc_auc`
- instead of modeling the stacked `POSTOP` data with `LogisticRegression` can try another model such as `RandomForest` or `SVM`
- in addition can also try using `PolynomialFeatures` transformation using `Scenario B` to see if any benefit from modeling the interaction of the `PREOP` predictions with the additional `POSTOP` features
- should examine the non-zero coeficients in `POSTOP` to get insight into what `POSTOP` features are useful in enhancing stroke prediction

## Step 3 ab - Just Preop Data RF

In [194]:
PREOP_RF_clf_pipe = Pipeline(steps=[
    ('get_PREOP_TREE_feature_matrix', PREOP_feature_matrix_TREE),
    ('Classifier', RandomForestClassifier())])

In [195]:
parameter_grid = {'Classifier__n_estimators': [315],
                  'Classifier__max_features': [0.1],
                  'Classifier__max_depth': [None],
                  'Classifier__min_samples_split': [50],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__random_state': [0]
                 }
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

In [196]:
gs = GridSearchCV(PREOP_RF_clf_pipe, #pipline
                  param_grid=parameter_grid, #param_grid
                  scoring=scoring_metrics, #loss functions
                  cv=5, #number of folds, default=`StratifiedKFold`
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  return_train_score=True,
                  n_jobs=-1,
                  verbose=5)

In [197]:
PRE_modelRF = gs.fit(X_train_all, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    7.6s remaining:   11.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.1s finished


In [198]:
gs_results = pd.DataFrame(PRE_modelRF.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                  ascending=False)
display_cols = ['param_Classifier__n_estimators',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)
gs_results_summary.rename(columns={'param_Classifier__n_estimators': 'n_estimators'},
                          inplace=True)

In [199]:
gs_results_summary

n_estimators  mean_test_roc_auc  mean_train_roc_auc  mean_test_f1  \
0          315            0.63201             0.99989           0.0   

   mean_test_precision  mean_test_recall  
0                  0.0               0.0

### Step 3b: Model and Evaluate Stacked Model Using `RandomForestClassifier`

#### Creating `POSTOP_feature_matrix_TREE` using `sklearn` `Pipelines` 

- can use similar coding pattern to assemble `POSTOP_feature_matrix_TREE` if needed
    - `POSTOP_numerical_features_pipeline` has been previously created above
    - would need to create `POSTOP_categorical_TREE_features_pipeline` -- THIS HAS NOT BEEN DONE YET
    - then use `FeatureUnion()` to assemble `POSTOP_feature_matrix_TREE`
    - similar to what was done above in using `Pipeline`s to create `X_train_PREOP`, `X_train_POSTOP` and corresponding `X_dev` and `X_test` 

In [200]:
POSTOP_categorical_TREE_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(POSTOP_categorical_TREE_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- using `FeatureUnion` combining using with already created `POSTOP_numerical_features_pipeline` to assemble `POSTOP_feature_matrix_TREE`

In [201]:
POSTOP_feature_matrix_TREE = FeatureUnion([('POSTOP_num_features', POSTOP_numerical_features_pipeline),
                                           ('POSTOP_cat_tree_features', POSTOP_categorical_TREE_features_pipeline)
                                          ])

#### NOTE - as we did above:

- `fit_transform` only on `X_train_all` for `X_train_POSTOP_TREE`
- `transform` on `X_dev_all` and `X_test_all` for `X_dev_POSTOP_TREE` and `X_test_POSTOP_TREE`
- assign compreleted `feature_matrix` to variable name if not using directly in a `Pipeline` with classifier step

#### Transforming `X_train_all` to get `X_train_POSTOP_TREE`

In [202]:
X_train_POSTOP_TREE = POSTOP_feature_matrix_TREE.fit_transform(X_train_all)

- validating

In [203]:
X_train_all.shape, X_train_POSTOP_TREE.shape, y_train.shape

((4765, 247), (4765, 62), (4765,))

- putting into a `DataFrame` for easy analysis

In [204]:
POSTOP_feature_TREE_col_names = POSTOP_numerical_col_names + POSTOP_categorical_TREE_col_names

In [205]:
len(POSTOP_feature_TREE_col_names), len(POSTOP_numerical_col_names), len(POSTOP_categorical_TREE_col_names)

(62, 15, 47)

In [206]:
X_train_POSTOP_TREE = pd.DataFrame(X_train_POSTOP_TREE,
                                   columns=POSTOP_feature_TREE_col_names)

In [207]:
X_train_POSTOP_TREE.head()

cperftime  cumulsatlft  cumulsatrt   dhcatm  ibdcryou  ibdffpu  ibdplatu  \
0   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   
1   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   
2   -0.00495     -0.07625    -0.08823  0.00954  -0.12586  0.55797  -0.35738   
3   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392   1.33035   
4   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   

   ibdrbcu  lwsthct  lwsttemp  perfustm  postcreat  prerso2lft  prerso2rt  \
0 -0.03049 -0.61060  -0.54267   2.99604    1.43861     0.02756    0.02721   
1 -0.03049  0.72886  -0.54267   0.34339   -0.60120     0.02756    0.02721   
2  0.79022  0.50561   0.15682   0.68165    0.00319     0.02756    0.02721   
3 -1.67189  1.84507  -1.34209  -0.33312   -0.14791     0.02756    0.02721   
4 -0.03049  0.05913   1.05617  -0.70699   -0.45010     0.02756    0.02721   

   xclamptm  canartstaort  canartstfem  canartstoth  canartstax  \
0   2.47082           1.0          0.0          0.0         0.0   
1  -0.11015           1.0          0.0          0.0         0.0   
2  -0.22335           1.0          0.0          0.0         0.0   
3  -0.67615           1.0          0.0          0.0         0.0   
4  -0.47239           1.0          0.0          0.0         0.0   

   cathbasassist  ceroxused  circarr  cofirstind  concalc  cperfutil  IABP  \
0            0.0        1.0      0.0         0.0      0.0        0.0   0.0   
1            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
2            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
3            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
4            0.0        0.0      0.0         0.0      0.0        0.0   0.0   

   ibldprod  imedeaca  imedtran  inoptee  ocarasd  ocarvsd  opocard  oponcard  \
0       0.0       1.0       0.0      1.0      0.0      0.0      1.0       0.0   
1       0.0       1.0       0.0      0.0      0.0      0.0      0.0       0.0   
2       1.0       1.0       0.0      0.0      1.0      0.0      1.0       0.0   
3       1.0       1.0       0.0      0.0      0.0      0.0      0.0       0.0   
4       0.0       1.0       0.0      1.0      0.0      0.0      0.0       0.0   

   unplao  unplav  unplmv  unplvad  vsmvpr  asmtascaa  cotafib  cotarrst  \
0     0.0     0.0     0.0      0.0     1.0        1.0      1.0       0.0   
1     0.0     0.0     0.0      0.0     1.0        1.0      1.0       0.0   
2     0.0     0.0     0.0      0.0     0.0        0.0      0.0       0.0   
3     0.0     0.0     0.0      0.0     0.0        0.0      1.0       0.0   
4     0.0     0.0     0.0      0.0     0.0        1.0      1.0       0.0   

   ecmo  mt30stat  opcab  opvalve  vadproc  vsavpr  vsmv  aortoccl  asmtaodx  \
0   0.0       1.0    1.0      1.0      0.0     0.0   1.0       1.0       1.0   
1   0.0       1.0    1.0      1.0      0.0     0.0   1.0       1.0       1.0   
2   0.0       1.0    1.0      1.0      0.0     0.0   1.0       1.0       0.0   
3   0.0       1.0    1.0      1.0      0.0     1.0   0.0       1.0       0.0   
4   0.0       1.0    1.0      1.0      0.0     1.0   0.0       1.0       1.0   

   cathbasassistind  cathbasassistwhen  cpbutil  cperftyp  ecmoind  ecmowhen  \
0               0.0                0.0      2.0       0.0      0.0       0.0   
1               0.0                0.0      2.0       0.0      0.0       0.0   
2               0.0                0.0      2.0       0.0      0.0       0.0   
3               0.0                0.0      2.0       0.0      0.0       0.0   
4               0.0                0.0      2.0       0.0      0.0       0.0   

   emergrsn  iabpind  iabpwhen  unplproc  urgntrsn  
0       0.0      0.0       0.0       0.0       0.0  
1       0.0      0.0       0.0       0.0       0.0  
2       0.0      0.0       0.0       0.0       5.0  
3       0.0      0.0       0.0       0.0       0.0  
4       0.0      0.0  

#### Creating `X_train_POSTOP_TREE_STACKED`
- adding vectors of `predict_probas` from `PREOP` models and `STS` models to `POSTOP_TREE` feature matrix

In [208]:
X_train_POSTOP_TREE_STACKED = X_train_POSTOP_TREE.copy()

In [209]:
X_train_POSTOP_TREE_STACKED.shape, X_train_POSTOP_TREE.shape

((4765, 62), (4765, 62))

In [210]:
X_train_POSTOP_TREE_STACKED['LR_PREOP_PROBA'] = lr_predict_probs[:, 1]
X_train_POSTOP_TREE_STACKED['RF_PREOP_PROBA'] = rf_predict_probs[:, 1]
#X_train_POSTOP_TREE_STACKED['STS_predstro'] = predstro_train

In [211]:
X_train_POSTOP_TREE_STACKED.head()

cperftime  cumulsatlft  cumulsatrt   dhcatm  ibdcryou  ibdffpu  ibdplatu  \
0   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   
1   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   
2   -0.00495     -0.07625    -0.08823  0.00954  -0.12586  0.55797  -0.35738   
3   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392   1.33035   
4   -0.00495     -0.07625    -0.08823  0.00954  -0.12586 -0.33392  -0.35738   

   ibdrbcu  lwsthct  lwsttemp  perfustm  postcreat  prerso2lft  prerso2rt  \
0 -0.03049 -0.61060  -0.54267   2.99604    1.43861     0.02756    0.02721   
1 -0.03049  0.72886  -0.54267   0.34339   -0.60120     0.02756    0.02721   
2  0.79022  0.50561   0.15682   0.68165    0.00319     0.02756    0.02721   
3 -1.67189  1.84507  -1.34209  -0.33312   -0.14791     0.02756    0.02721   
4 -0.03049  0.05913   1.05617  -0.70699   -0.45010     0.02756    0.02721   

   xclamptm  canartstaort  canartstfem  canartstoth  canartstax  \
0   2.47082           1.0          0.0          0.0         0.0   
1  -0.11015           1.0          0.0          0.0         0.0   
2  -0.22335           1.0          0.0          0.0         0.0   
3  -0.67615           1.0          0.0          0.0         0.0   
4  -0.47239           1.0          0.0          0.0         0.0   

   cathbasassist  ceroxused  circarr  cofirstind  concalc  cperfutil  IABP  \
0            0.0        1.0      0.0         0.0      0.0        0.0   0.0   
1            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
2            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
3            0.0        0.0      0.0         0.0      0.0        0.0   0.0   
4            0.0        0.0      0.0         0.0      0.0        0.0   0.0   

   ibldprod  imedeaca  imedtran  inoptee  ocarasd  ocarvsd  opocard  oponcard  \
0       0.0       1.0       0.0      1.0      0.0      0.0      1.0       0.0   
1       0.0       1.0       0.0      0.0      0.0      0.0      0.0       0.0   
2       1.0       1.0       0.0      0.0      1.0      0.0      1.0       0.0   
3       1.0       1.0       0.0      0.0      0.0      0.0      0.0       0.0   
4       0.0       1.0       0.0      1.0      0.0      0.0      0.0       0.0   

   unplao  unplav  unplmv  unplvad  vsmvpr  asmtascaa  cotafib  cotarrst  \
0     0.0     0.0     0.0      0.0     1.0        1.0      1.0       0.0   
1     0.0     0.0     0.0      0.0     1.0        1.0      1.0       0.0   
2     0.0     0.0     0.0      0.0     0.0        0.0      0.0       0.0   
3     0.0     0.0     0.0      0.0     0.0        0.0      1.0       0.0   
4     0.0     0.0     0.0      0.0     0.0        1.0      1.0       0.0   

   ecmo  mt30stat  opcab  opvalve  vadproc  vsavpr  vsmv  aortoccl  asmtaodx  \
0   0.0       1.0    1.0      1.0      0.0     0.0   1.0       1.0       1.0   
1   0.0       1.0    1.0      1.0      0.0     0.0   1.0       1.0       1.0   
2   0.0       1.0    1.0      1.0      0.0     0.0   1.0       1.0       0.0   
3   0.0       1.0    1.0      1.0      0.0     1.0   0.0       1.0       0.0   
4   0.0       1.0    1.0      1.0      0.0     1.0   0.0       1.0       1.0   

   cathbasassistind  cathbasassistwhen  cpbutil  cperftyp  ecmoind  ecmowhen  \
0               0.0                0.0      2.0       0.0      0.0       0.0   
1               0.0                0.0      2.0       0.0      0.0       0.0   
2               0.0                0.0      2.0       0.0      0.0       0.0   
3               0.0                0.0      2.0       0.0      0.0       0.0   
4               0.0                0.0      2.0       0.0      0.0       0.0   

   emergrsn  iabpind  iabpwhen  unplproc  urgntrsn  LR_PREOP_PROBA  \
0       0.0      0.0       0.0       0.0       0.0         0.39474   
1       0.0      0.0       0.0       0.0       0.0         0.42219   
2       0.0      0.0       0.0       0.0       5.0         0.65027   
3      

In [212]:
X_train_POSTOP_TREE_STACKED.shape

(4765, 64)

- instantiating a `RandomForestClassifier()` object
- code pattern from line 169 - `GridSearchCV`

In [213]:
rf_clf = RandomForestClassifier(class_weight='balanced',
                                random_state=0)

In [214]:
hyper_params = {'n_estimators': [100, 200, 250, 300],          # default is `10`
                'max_features': ['auto', 0.10, 0.25, 0.50],    # default is square root of `n_features`    
                'max_depth': [2, 5, 10, None],                 # default is `None`
                'min_samples_split': [2, 5, 10]}               # default is `2`

In [215]:
gs = GridSearchCV(rf_clf, # estimator
                  hyper_params, # param_grid
                  scoring=scoring_metrics, # loss functions
                  cv=5, 
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  verbose=2,
                  return_train_score=True,
                  n_jobs=-1)

In [222]:
rf_gs_stacked_model = gs.fit(X_train_POSTOP_TREE_STACKED, 
                             y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   38.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 51.7min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 78.5min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 183.1min finished


- `GridSearchCV` takes approximately 90 minutes to run

#### Saving `rf_gs_stacked_model`

In [240]:
joblib.dump(rf_gs_stacked_model, 'rf_gs_stacked_model_cabgValve_nosts_12_2.sav')

['rf_gs_stacked_model_cabgValve_nosts_12_2.sav']

#### Loading saved `rf_gs_stacked_model`

In [203]:
#rf_gs_stacked_model = joblib.load('rf_gs_stacked_model_11_17.sav')

- analyzing results

In [232]:
rf_gs_stacked_model.best_params_

{'max_depth': None,
 'max_features': 0.1,
 'min_samples_split': 10,
 'n_estimators': 300}

In [233]:
gs_results = pd.DataFrame(rf_gs_stacked_model.cv_results_)

In [234]:
gs_results = gs_results.sort_values(by=['mean_test_roc_auc'],
                                           ascending=False)

In [235]:
display_cols = ['params',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [236]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [237]:
gs_results_summary.head()

params  \
0  {'max_depth': None, 'max_features': 0.1, 'min_samples_split': 10, 'n_estimators': 300}   
1   {'max_depth': 2, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 100}   
2   {'max_depth': 2, 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 100}   
3  {'max_depth': 2, 'max_features': 'auto', 'min_samples_split': 10, 'n_estimators': 100}   
4   {'max_depth': 2, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 250}   

   mean_test_roc_auc  mean_train_roc_auc  mean_test_f1  mean_test_precision  \
0            0.66332             1.00000       0.00000              0.00000   
1            0.66228             0.75555       0.10528              0.05933   
2            0.66225             0.75552       0.10511              0.05923   
3            0.66225             0.75551       0.10511              0.05923   
4            0.66211             0.76110       0.10140              0.05731   

   mean_test_recall  
0           0.00000  
1           0.46850  
2           0.46850  
3           0.46850  
4           0.44019

#### Key Takeaways
- better than `STS` alone, but results slightly below that of `LogisticRegression()`

## Results on test set

In [216]:
def calcTestROC(model, X_test, y_test):
    pred_prob = model.predict_proba(X_test)[:,1]
    return roc_auc_score(y_test, pred_prob)

In [217]:
calcTestROC(PREPOST_modelA, X_test_all, y_test ) #log reg on pre+post data

0.5914396887159533

In [218]:
calcTestROC(PRE_modelA, X_test_all, y_test) #log reg on pre data

0.5583657587548638

In [219]:
calcTestROC(PRE_modelRF, X_test_all, y_test ) #RF on pre data

0.6285262645914397

In [220]:
#need to create X_test_POSTOP_STACKED
X_test_POSTOP_STACKED = X_test_POSTOP.copy()
X_test_POSTOP_STACKED['LR_PREOP_PROBA'] = PRE_modelA.predict_proba(X_test_all)[:, 1]
X_test_POSTOP_STACKED['RF_PREOP_PROBA'] = PRE_modelRF.predict_proba(X_test_all)[:, 1]

In [221]:
calcTestROC(PREPOST_modelB, X_test_POSTOP_STACKED, y_test ) #log reg on pre+post stacked

0.6205009727626459

In [238]:
#need to create X_test_POSTOP_TREE_STACKED
X_test_POSTOP_TREE = POSTOP_feature_matrix_TREE.transform(X_test_all)
X_test_POSTOP_TREE = pd.DataFrame(X_test_POSTOP_TREE,
                                   columns=POSTOP_feature_TREE_col_names)
X_test_POSTOP_TREE_STACKED = X_test_POSTOP_TREE.copy()
X_test_POSTOP_TREE_STACKED['LR_PREOP_PROBA'] = PRE_modelA.predict_proba(X_test_all)[:, 1]
X_test_POSTOP_TREE_STACKED['RF_PREOP_PROBA'] = PRE_modelRF.predict_proba(X_test_all)[:, 1]

In [239]:
calcTestROC(rf_gs_stacked_model, X_test_POSTOP_TREE_STACKED, y_test ) #RF on pre+post stacked

0.6029912451361867

In [241]:
round(roc_auc_score(y_test,predstro_test),5)

0.72112

In [242]:
round(roc_auc_score(y_train,predstro_train),5)

0.67539

### `Polynomial` Transformation on Stacked Model
- to see if adding interaction terms between `PREOP` models predicted probabilities and `POSTOP` features can improve results

- instantiating `Pipeline` that will include `Polynomial` transformation of `X_train_POSTOP_STACKED`

In [210]:
poly_lr_model_pipe = Pipeline(steps=[
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('Classifier', LogisticRegression())
    ])

- defining a parameter grid for `GridSearchCV`

In [211]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.0001, 0.001, 0.01, 0.10, 0.50, 1.0, 10.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

- create `GridSearchCV` object

In [212]:
gs = GridSearchCV(poly_lr_model_pipe, # estimator
                  param_grid=parameter_grid, # param_grid
                  scoring=scoring_metrics, # loss functions
                  cv=5,
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  verbose=2,
                  return_train_score=True,
                  n_jobs=-1)

- fitting `GridSearchcV`

- `GridSearchCV` takes over 2 full hours to run

#### Saving `poly_log_reg_lasso`

In [213]:
#joblib.dump(poly_log_reg_lasso, 'poly_log_reg_lasso_model_11_17.sav')

#### Loading `poly_log_reg_lasso`

In [214]:
poly_log_reg_lasso = joblib.load('poly_log_reg_lasso_model_11_17.sav')

- examining results

In [215]:
poly_log_reg_lasso.best_params_

{'Classifier__C': 0.01,
 'Classifier__class_weight': 'balanced',
 'Classifier__penalty': 'l1',
 'Classifier__random_state': 0,
 'Classifier__solver': 'liblinear'}

In [216]:
gs_results = pd.DataFrame(poly_log_reg_lasso.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                      ascending=False)

In [217]:
display_cols = ['params',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [218]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [219]:
gs_results_summary

params  \
0  {'Classifier__C': 0.01, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cl...   
1  {'Classifier__C': 0.001, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'C...   
2  {'Classifier__C': 0.1, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cla...   
3  {'Classifier__C': 0.5, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cla...   
4  {'Classifier__C': 0.0001, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', '...   
5  {'Classifier__C': 1.0, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cla...   
6  {'Classifier__C': 10.0, 'Classifier__class_weight': 'balanced', 'Classifier__penalty': 'l1', 'Cl...   

   mean_test_roc_auc  mean_train_roc_auc  mean_test_f1  mean_test_precision  \
0            0.69959             0.78409       0.06840              0.03647   
1            0.65922             0.66717       0.05326              0.02775   
2            0.61247             0.90305       0.06275              0.03429   
3            0.54727             0.94213       0.05680              0.03157   
4            0.53768             0.54504       0.03347              0.01702   
5            0.52516             0.95176       0.05498              0.03078   
6            0.48404             0.96689       0.05166              0.02937   

   mean_test_recall  
0           0.54979  
1           0.66316  
2           0.36941  
3           0.28345  
4           1.00000  
5           0.25774  
6           0.21479

#### Key Takeways
- `PolynomialTransformation` actually had worse results than non-transformed `LogisticRegression`
- Results were in-line with `STS` model

## CODE BELOW IS USEFUL FOR DIAGNOSTIC CHARTS ON MODEL SCORING

### Need to modify and adapt for code above given changes in variable names

In [220]:
break

SyntaxError: 'break' outside loop (<ipython-input-220-6aaf1f276005>, line 4)

# ---------------------------------------------------------------------------------------------

## Visualizing the Effect of `LogisticRegression()` Hyperparameter Values
- See Albon `11.13` pages 205-207
- range of values for `C` - inverse of regularization strength

In [ ]:
param_values = np.linspace(0.001, 0.05, 25)

- calculating `roc_auc` on training and test set using `param_range` for `C`

In [ ]:
train_scores, test_scores = validation_curve(LogisticRegression(penalty='l1',
                                                                class_weight='balanced',
                                                                random_state=0,
                                                                solver='liblinear'), #classifier
                                             X_train, #feature matrix
                                             y_train, #target vector
                                             param_name='C', #hyperparameter to examine,
                                             param_range=param_values, #range of hyperparameter values
                                             cv=5, #number of folds, default=`StratifiedKFold`
                                             scoring='roc_auc', #scoring metric
                                             n_jobs=-1, #use all computer cores
                                             verbose=5) 

- calculate `mean` and `standard deviation` for `training` set scores

In [ ]:
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

- calculate `mean` and `standard deviation` for `test` set scores

In [ ]:
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

- Plotting

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')

# plot mean accuracy scores for training and test sets
plt.plot(param_values,
         train_mean,
         label='Training Score',
         color='midnightblue')

plt.plot(param_values,
         test_mean,
         label='Cross Validation Score',
         color='darkviolet')

# plot accuracy bands for training and test sets
plt.fill_between(param_values, 
                 train_mean - train_std,
                 train_mean + train_std,
                 color='lightsteelblue')

plt.fill_between(param_values,
                 test_mean - test_std,
                 test_mean + test_std,
                 color='thistle')

# create plot
plt.title('Validation Curve for Logistic Regression with l1 Regularization')
plt.xlabel('C')
plt.ylabel('AUCROC')
plt.legend(loc='lower right')
plt.show()       

In [ ]:
validation_summary = pd.DataFrame({'C': param_values,
                                   'mean_test_roc_auc': test_mean,
                                   'mean_train_roc_auc': train_mean})

In [ ]:
validation_summary[validation_summary['mean_test_roc_auc'] == np.max(validation_summary['mean_test_roc_auc'])]

#### How does this tuned model compare to the baseline `STS` Model

In [ ]:
round(roc_auc_score(y_train, predstro_train), 5)

#### Key Takeaways
- Fine tuning of hyperparmeter `C` results in `C=0.0275`
- Results in-line (but still lower) than `STS` Model

## Precision-Recall Curve for Tuned Logistic Regression Model
- `Hands On Machine Learning with Scikit-Learn & Tensorflow` by Aurelien Geron pages 83-93
#### How `cross_val_predict` works:
- `cross_val_predict` returns for each element in the input, the prediction that was obtained for that element when it was in the `test` set

- generating vector of probabilities of stroke for cross validated training set

In [ ]:
lr_predict_probs = cross_val_predict(LogisticRegression(penalty='l1',
                                                        C=0.0275,
                                                        class_weight='balanced',
                                                        random_state=0,
                                                        solver='liblinear'),
                                     X_train,
                                     y_train,
                                     cv=5,
                                     method='predict_proba',
                                     n_jobs=-1,
                                     verbose=5)

- using `lr_predict_probs` can compute precision and recall for all possible thresholds using the `precision_recall_curve` function
- returns an `numpy.ndarray` with `shape` `(n, 2)` as it returns a probability per class `(0, 1)`
- since we are interested in the `1` class, need to carve our the second column of the `numpy.ndarray`
- add `[:,1]`

In [ ]:
type(lr_predict_probs)

In [ ]:
lr_predict_probs.shape

In [ ]:
lr_predict_probs[0:5]

In [ ]:
lr_predict_probs[0:5, 1]

#### Generating `precision_recall_curve`

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(y_train,
                                                         lr_predict_probs[:, 1])

- `precision` values such that element `i` is the the `precision` of predictions with `score>=threholds[i]` and the last element is `1`

In [ ]:
precisions[0:5] # first 5 elements of the numpy array

In [ ]:
precisions[-5:] # last 5 elements of the numpy array

- decreasing `recall` values such that element `i` is the the `precision` of predictions with `score>=threholds[i]` and the last element is `0`

In [ ]:
recalls[0:5]

In [ ]:
recalls[-5:]

- increasing `thresholds` on the decision function used to compute `precision` and `recall`

In [ ]:
thresholds[0:5]

In [ ]:
thresholds[-5:]

### Plotting `precision` and `recall` as functions of `threshold`

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], 'b--', label='Precision')
    plt.plot(thresholds, recalls[:-1], 'g-', label='Recall')
    plt.title('Precision and Recall versus Threshold')
    plt.xlabel('Threshold')
    plt.legend(loc='upper right')
    plt.ylim([0, 1])

### Precision and Recall versus Threshold for Logistic Regression Model

In [ ]:
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

- helper function

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx], idx

- examining specific `threshold`, `Recall` and `Precision` trade-offs
- suppose you wanted a `recall` of `0.80`, what `threshold` will you need and what will the `precision` be?

In [ ]:
find_nearest(recalls, 0.80)

In [ ]:
print('{0:15} {1:5f}'.format('Recall:', recalls[14806]))
print('{0:15} {1:5f}'.format('Precision:', precisions[14806]))
print('{0:15} {1:5f}'.format('Threshold:', thresholds[14806]))

- see `Geron` page 90 for additional details

### Precision and Recall versus Threshold for STS Model

In [ ]:
sts_precisions, sts_recalls, sts_thresholds = precision_recall_curve(y_train,
                                                                     predstro_train)

In [ ]:
plot_precision_recall_vs_threshold(sts_precisions, sts_recalls, sts_thresholds)
plt.show()

## Precision-Recall Curves
- `Introduction to Machine Learning with Python` by Andreas C. Muller and Sarah Guido pages 289-296
- the closer a `precision-recall curve` stays to the upper-right corner, the better classifier

In [ ]:
def plot_precision_recall_curve(precisions, recalls):
    plt.plot(precisions,
             recalls,
             color='blue',
             label='Precision-Recall Curve')
    plt.title('Precision-Recall Curve')
    plt.ylabel('Recall')
    plt.xlabel('Precision')
    plt.legend(loc='upper right')

### `Precision-Recall Curve` for `LogisticRegression` Model

In [ ]:
plot_precision_recall_curve(precisions, recalls)
plt.show()

#### Area Under the `Precision-Recall Curve` for `LogisticRegression` Model

In [ ]:
round(auc(recalls, precisions), 5)

### `Precision-Recall Curve` for `STS` Model

In [ ]:
plot_precision_recall_curve(sts_precisions, sts_recalls)
plt.show()

#### Area Under the `Precision-Recall Curve` for `STS` Model

In [ ]:
round(auc(sts_recalls, sts_precisions), 5)

#### Key Takeaways
- `STS` model is superior to `LogisticRegression` model looking at `Precision-Recall Curves` and `PR AUC`

### Comparing `Logistic Regression` and `STS` Model `Precision-Recall` Curves

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')

# plotting `LogisticRegression` Precision-Recall Curve
plt.plot(precisions,
         recalls,
         'b--',
         label='Logistic Regression Model')

# plotting `STS Model` Precision-Recall Curve
plt.plot(sts_precisions,
         sts_recalls,
         'g-',
         label='STS Model')

plt.title('Logistic Regression, STS Model Precision-Recall Curves')
plt.ylabel('Recall')
plt.xlabel('Precision')
plt.legend(loc='upper right')
plt.xlim([0, 0.10]) # cut off values > 0.10 because they were essentially zero
plt.show()

#### Key Takeaways
- plot clearly shows `STS` model outperperformance over `LogisticRegression`
- can use basic code here to plot `Precision-Recall Curves` from other models on the same plot for comparison

## Receiver Operating Characteristic (`ROC`) Curves
- Albon `11.5` pages 189-192
- a classifier that predicts every observation correctly would look like the solid `gray` line in the plotted `ROC` curves below
- the solid `gray` line goes straight up to the top immediately
- a classifier that predicts at random will appear as the diagonal line
- the better the model, the closer it is to the solid `gray` line
- the `ROC` curve represents the respective `TPR` and `FPR` for every probability threshold

- creating `true` and `false` probabilities

In [ ]:
false_positive_rate, true_positive_rate, threshold = roc_curve(y_train,
                                                               lr_predict_probs[:, 1])

- `plot_roc_curve` function

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.title('Receiver Operating Curve (ROC)')
    plt.plot(fpr, tpr)
    plt.plot([0, 1], ls='--')
    plt.plot([0, 0],
             [1, 0],
             c='0.7'),
    plt.plot([1, 1], c='0.7')
    plt.ylabel('True Positive Rate (Recall)')
    plt.xlabel('False Positive Rate')

#### `LogisiticRegression` Model `ROC` Curve

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')
plot_roc_curve(false_positive_rate, true_positive_rate)
plt.show()

In [ ]:
round(roc_auc_score(y_train, lr_predict_probs[:, 1]), 5)

- suppose you want `recall` in a certain range - what is the `false positive rate` you would have to accept?

In [ ]:
lr_roc_curve = pd.DataFrame({'Recall': true_positive_rate,
                             'FPR': false_positive_rate})

- assume you want `recall` around `0.80`

In [ ]:
lr_roc_curve[lr_roc_curve['Recall'].between(0.799, 0.801)]

- could have also used the `find_nearest` helper function above

In [ ]:
lr_roc_curve[lr_roc_curve['Recall'] == (find_nearest(lr_roc_curve['Recall'], 0.80)[0])]

- suppose you want to know `recall` and `FPR` for a given threshold

In [ ]:
np.where(threshold == 0.10)[0]

In [ ]:
print('{0:15} {1:5f}'.format('Threshold:', threshold[11]))
print('{0:15} {1:5f}'.format('Recall:', true_positive_rate[11]))
print('{0:15} {1:5f}'.format('FPR:', false_positive_rate[11]))

#### `STS` Model `ROC` Curve

In [ ]:
sts_fpr, sts_tpr, threshold = roc_curve(y_train,
                                        predstro_train)

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')
plot_roc_curve(sts_fpr, sts_tpr)
plt.show()

In [ ]:
round(roc_auc_score(y_train, predstro_train), 5)

#### Key Takeaways
- `STS` Model slightly outperforms `LogisticRegression`